---
# OpenCL Programming in C
---

This notebook contains an introduction to OpenCL programming in C. For detailed coverage, the Khronos Group's documentation is a good source:

- [Khronos Group OpenCL](https://www.khronos.org/opencl)
- [OpenCL Specification](https://registry.khronos.org/OpenCL/specs/3.0-unified/html/OpenCL_API.html)
- [OpenCL SDK](https://github.com/KhronosGroup/OpenCL-SDK)
- [OpenCL Guide](https://github.com/KhronosGroup/OpenCL-Guide?tab=readme-ov-file)

**Note! If you don't have an OpenCL-enabled device on your system**

- You can run this notebook in Google CoLab.
  - Skip ahead to [1.1 Running the Notebook Locally or on Google CoLab](#11-running-the-notebook-locally-or-on-google-coLab)

**Note! If you are on Windows**

- Make sure you have installed Visual Studio or the Build Tools for Visual Studio.
- Make sure you have started VSCode (`code .`) from within a `Visual Studio Developer Command Prompt` to set necessary environment variables.
  - This is required when using the MicroSoft Visual C/C++ (MSVC) compiler `cl.exe` in VSCode on Windows.
- If you are using PowerShell as your default shell in VSCode, your default PowerShell profile file `profile.ps1` might not be digitally signed.
  - This will lead to errors when you compile C code in VSCode.
  - If so, you can fix this error by executing either of the two PowerShell commands below:
    - `Rename-Item "$env:USERPROFILE\Documents\WindowsPowerShell\profile.ps1" -NewName "profile.ps1.bak"`
    - `Set-ExecutionPolicy RemoteSigned -Scope CurrentUser`

This notebook covers:

- [1. Prerequisites](#1-prerequisites) 
  - [1.1 Running the Notebook Locally or on Google CoLab](#11-running-the-notebook-locally-or-on-google-coLab)
  - [1.2 Operating System and VSCode Shell](#12-operating-system-and-vscode-shell)
  - [1.3 C Compiler (`gcc`, `clang`, `cl`)](#13-c-compiler-gcc-clang-cl)
  - [1.4 OpenCL Library and Header Files](#14-opencl-library-and-header-files)
  - [1.5 Configuring `tasks.json`, `launch.json` and `c_cpp_properties.json`](#15-configuring-tasksjson-launchjson-and-c_cpp_propertiesjson)
  - [1.6 Create the File `tasks.json`](#16-create-the-file-tasksjson)
  - [1.7 Create the File `launch.json`](#17-create-the-file-launchjson)
  - [1.8 Create the File `c_cpp_properties.json`](#18-create-the-file-c_cpp_propertiesjson)
  - [1.9 VSCode Extensions](#19-vscode-extensions)
  - [1.10 Using Built-in Cell Magic `%%writefile`](#110-using-built-in-cell-magic-writefile)
  - [1.11 Compiling and Executing an OpenCL Program from a Notebook Code Cell](#111-compiling-and-executing-an-opencl-program-from-a-notebook-code-cell)
  - [1.12 Compiling and Debugging a Single-file OpenCL Program](#112-compiling-and-debugging-a-single-file-opencl-program)
  - [1.13 Compiling and Debugging a Multi-file OpenCL Program](#113-compiling-and-debugging-a-multi-file-opencl-program)
- [2. OpenCL Basics](#2-opencl-basics)
  - [2.1 Listing OpenCL-enabled Devices and Properties](#21-listing-opencl-enabled-devices-and-properties)
  - [2.2 Hello World in Host Code (CPU)](#22-hello-world-in-host-code-cpu)
  - [2.3 Hello World in Device Code (GPU)](#23-hello-world-in-device-code-gpu)
  - [2.4 NDRange (Global Size), Work Groups, Work Items, Devices, CUs, and PEs](#24-ndrange-global-size-work-groups-work-items-devices-cus-and-pes)
  - [2.5 Error Checking](#25-error-checking)
  - [2.6 Measuring Execution Time on the Host (CPU) and on the Device (GPU)](#26-measuring-execution-time-on-the-host-cpu-and-on-the-device-gpu)
  - [2.7 Shared Memory and Thread Synchronization on the Device (GPU)](#27-shared-memory-and-thread-synchronization-on-the-device-gpu)
  - [2.8 Constant Memory on the Device (GPU)](#28-constant-memory-on-the-device-gpu)
- [3. Sample Problems](#3-sample-problems)
  - [3.1 1D Vector Addition on the Host (CPU)](#31-1d-vector-addition-on-the-host-cpu)
  - [3.2 1D Vector Addition on the Device (GPU)](#32-1d-vector-addition-on-the-device-gpu)
  - [3.3 1D Convolution on the Host (CPU)](#33-1d-convolution-on-the-host-cpu)
  - [3.4 1D Convolution on the Device (GPU)](#34-1d-convolution-on-the-device-gpu)
  - [3.5 2D Convolution on the Host (CPU)](#35-2d-convolution-on-the-host-cpu)
  - [3.6 2D Convolution on the Device (GPU)](#36-2d-convolution-on-the-device-gpu)
- [4. Cleanup](#4-cleanup)

---
# 1. Prerequisites
---

## 1.1 Running the Notebook Locally or on Google CoLab

Let's use the tool `clinfo` to list OpenCL-enabled devices on your system.
- If you don't have `clinfo` installed, first install it on your system:
  
  **Linux**
  
  - Install `clinfo` with the [Advanced Package Tool](https://www.debian.org/doc/manuals/debian-faq/pkgtools.en.html#apt-get)
    
    ```bash
    sudo apt install -y clinfo
    ```

  **Mac**
  
  - Install `clinfo` with [Homebrew](https://brew.sh)
    
    ```bash
    brew install clinfo
    ```

  **Windows**
  
  - Download `clinfo.exe` from here: https://github.com/engineer1109/clinfo/releases/tag/v1.0
  - Add it to your `PATH` environment variable (or just place it in your VSCode workspace folder).

- Then run the cell below to check if you have an OpenCL-enabled device on your system.

In [1]:
!clinfo

Number of platforms                               1
  Platform Name                                   NVIDIA CUDA
  Platform Vendor                                 NVIDIA Corporation
  Platform Version                                OpenCL 3.0 CUDA 12.8.90
  Platform Profile                                FULL_PROFILE
  Platform Extensions                             cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_khr_fp64 cl_khr_3d_image_writes cl_khr_byte_addressable_store cl_khr_icd cl_khr_gl_sharing cl_nv_compiler_options cl_nv_device_attribute_query cl_nv_pragma_unroll cl_nv_copy_opts cl_khr_gl_event cl_nv_create_buffer cl_khr_int64_base_atomics cl_khr_int64_extended_atomics cl_nv_kernel_attribute cl_khr_device_uuid cl_khr_pci_bus_info cl_khr_external_semaphore cl_khr_external_memory cl_khr_external_semaphore_opaque_fd cl_khr_external_memory_opaque_fd cl_khr_semaphore
  Platform Extensions

### Inspect the output from the cell above:
- If you see a `Number of platforms` listed above with a value of at least `1`.
  - Skip to [1.2 Operating System and VSCode Shell](#12-operating-system-and-vscode-shell)
- If you don't see a `Number of platforms` listed above with a value of at least `1`, follow the instructions below.

  1. Click the icon below to open the notebook in Google CoLab.
     
     [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/paga-hb/C1PD2C_2025/blob/main/notebooks/opencl.ipynb)

  2. When the notebook opens in CoLab, choose `File -> Save a copy in Drive` from the main menu.
  3. Choose `Runtime -> Change runtime type` from the main menu, select `TP4 GPU` as the hardware accelerator, and click the `Save` button.
  4. In a notebook cell, run the following code:

      ```c
      !sudo apt install -y gdb
      ```

  5. When the cell stops executing, continue with [1.2 Operating System and VSCode Shell](#12-operating-system-and-vscode-shell)

---

## 1.2 Operating System and VSCode Shell

We are going to compose JSON configuration files for VSCode, so let's collect some information about your environment.

- Let's start by finding out what OS you are on and what default shell you are using in VSCode.

**Linux/Mac/CoLab**

- Run the cell below.

**Windows**

- Find out (or change) which shell you are using in VSCode.
  - Open the Command Palette: `Ctrl + Shift + P`
  - Enter the text (and press `<Enter>`): `Preferences: Open Settings (UI)`
  - In the search field, enter the text: `terminal.integrated.defaultProfile.windows`
  - Choose the tab `User` or `Workspace` (`User` are global settings, `Workspace` only applies to the current workspace)
  - Click the link: `Edit in settings.json`
  - Set your desired shell:
    - `"terminal.integrated.defaultProfile.windows": "PowerShell"`
    - `"terminal.integrated.defaultProfile.windows": "Command Prompt"`
- Choose your VSCode shell in the cell below.
  - If you are using Powershell:
    - Comment the row `windows_shell = "cmd"`
    - Uncomment the row `windows_shell = "powershell"`
- Run the cell below.

In [1]:
windows_shell_name = "cmd"
#windows_shell_name = "powershell"

import platform, os
os_name = platform.system()
if os_name == "Darwin":
    os_name = "osx"
os_name = os_name.lower()

print(f"{'Operating System (OS)':<21} : {os_name}")
if os_name == 'windows':
    windows_shell_path = !where {windows_shell_name}
    windows_shell_path = windows_shell_path[0]
    windows_shell_name = os.path.basename(windows_shell_path)
    print(f"{'Windows Shell Name':<21} : {windows_shell_name}")
    print(f"{'Windows Shell Path':<21} : {windows_shell_path}")

Operating System (OS) : linux


---
## 1.3 C Compiler (`gcc`, `clang`, `cl`)

To avoid full paths to the C compiler and debugger in the JSON configuration files, make sure the path to the C compiler is in your `PATH` environment variable.

**CoLab**
- Run the cell below.

**Linux/Mac/Windows**
- In the cell below, choose the installed C compiler you want to use  
  - If you are using `cl` (the C/C++ compiler, part of Microsoft Visual Studio build tools).
    - Make sure you have launched VSCode from within a `Developer Command Prompt for VS`.
      - Search in your Start Menu for `Developer Command Prompt for VS` (the version depends on your installed Visual Studio version).
      - Open it => it launches a command prompt with all environment variables (paths, includes, libs) configured to run `cl.exe` and other build tools.
      - Open VSCode from the command prompt: `code .`
    - Comment the row `c_compiler = "gcc"`
    - Uncomment the row `c_compiler = "cl"`
  - If you are using `clang` (the C compiler, part of the LLVM project).
    - Comment the row `c_compiler = "gcc"`
    - Uncomment the row `c_compiler = "clang"`
  - If you are using `gcc` (GNU Compiler Collection), you're all set.
- Run the cell below to get the path to the C compiler.
- If nothing shows up, you need to install a C compiler (and/or make sure the C compiler is in your `PATH` environment variable).

In [3]:
c_compiler = "gcc"
#c_compiler = "clang"
#c_compiler = "cl"

import os
if os_name == 'windows':
    c_compiler_path = !where {c_compiler}
else:
    c_compiler_path = !which {c_compiler}
c_compiler_path = c_compiler_path[0]
c_compiler_name = os.path.basename(c_compiler_path)

if c_compiler == 'cl':
    c_debugger_name = "cdb.exe"
    c_debugger_path = "<integrated>"
if c_compiler == "gcc":
    c_debugger_name = "gdb"
if c_compiler == "clang":
    c_debugger_name = "lldb"

if os_name == 'windows':
    if c_compiler != 'cl':
        c_debugger_path = !where {c_debugger_name}
else:
    c_debugger_path = !which {c_debugger_name}

if c_compiler != 'cl':
    c_debugger_path = c_debugger_path[0]
    c_debugger_name = os.path.basename(c_debugger_path)

print(f"{'C Compiler Name':<15} : {c_compiler_name}")
print(f"{'C Compiler Path':<15} : {c_compiler_path}")
print(f"{'C Debugger Name':<15} : {c_debugger_name}")
print(f"{'C Debugger Path':<15} : {c_debugger_path}")

C Compiler Name : gcc
C Compiler Path : /usr/bin/gcc
C Debugger Name : gdb
C Debugger Path : /usr/bin/gdb


---
## 1.4 OpenCL Library and Header Files

Let's find out where your OpenCL library and header files are located on your system.

- Run the cell below to get the path to OpenCL's library and header files.
- If nothing shows up, you need to install the OpenCL SDK for at least one device on your computer (and/or make sure environment variables are set up correctly).

In [4]:
if os_name == "linux":
    !find /usr -name cl.h 2>/dev/null
    !find /usr -name libOpenCL.so 2>/dev/null

if os_name == "osx":
    print("/System/Library/Frameworks/OpenCL.framework/Headers/")
    print("/System/Library/Frameworks/OpenCL.framework/OpenCL/")

if os_name == "windows":
    shell_path = !echo %COMSPEC%
    if os.path.basename(shell_path[0]) == "powershell.exe":
        !Get-ChildItem -Recurse -Force -Filter "cl.h" -Path "$env:USERPROFILE" -ErrorAction SilentlyContinue
        !Get-ChildItem -Recurse -Force -Filter "cl.h" -Path "$env:PROGRAMFILES" -ErrorAction SilentlyContinue
        !Get-ChildItem -Recurse -Force -Filter "OpenCl.lib" -Path "$env:USERPROFILE" -ErrorAction SilentlyContinue
        !Get-ChildItem -Recurse -Force -Filter "OpenCl.lib" -Path "$env:PROGRAMFILES" -ErrorAction SilentlyContinue
    else:
        !dir /s /b "%USERPROFILE%\cl.h"
        !dir /s /b "%ProgramFiles%\cl.h"
        !dir /s /b "%USERPROFILE%\OpenCl.lib"
        !dir /s /b "%ProgramFiles%\OpenCl.lib"

/usr/include/CL/cl.h
/usr/local/cuda-12.8/targets/x86_64-linux/include/CL/cl.h
/usr/local/cuda-12.8/targets/x86_64-linux/lib/libOpenCL.so
/usr/lib/x86_64-linux-gnu/libOpenCL.so


### From the paths listed above:
- Choose one path to the header files (`.h`) listed above (enter the full path as listed above).
- Choose one path to the library file (`.so` or `.lib`) listed above (enter the full path as listed above).
- Enter the paths in the cell below.
- Run the cell below.

In [5]:
opencl_include_path = r"/usr/local/cuda-12.8/targets/x86_64-linux/include/CL/cl.h"
opencl_lib_path = r"/usr/local/cuda-12.8/targets/x86_64-linux/lib/libOpenCL.so"

import os
opencl_include_dir = os.path.dirname(opencl_include_path)
if os.path.basename(opencl_include_dir) == "CL":
    opencl_include_dir = f'"{os.path.dirname(opencl_include_dir)}"'

opencl_lib_dir = f'"{os.path.dirname(opencl_lib_path)}"'
print(f"{'OpenCL Include Path':<19} : {opencl_include_dir}")
print(f"{'OpenCL Lib Path':<19} : {opencl_lib_dir}")

OpenCL Include Path : "/usr/local/cuda-12.8/targets/x86_64-linux/include"
OpenCL Lib Path     : "/usr/local/cuda-12.8/targets/x86_64-linux/lib"


---
## 1.5 Configuring `tasks.json`, `launch.json`, and `c_cpp_properties.json`

- To develop OpenCL programs in C with VSCode, we need to configure three VSCode workspace configuration files.
  - In the file `tasks.json` we can configure various tasks, such as build tasks for compiling OpenCL programs in C with our chosen C compiler.
  - In the file `launch.json` we can configure various debug options, such as debugging C programs with our chosen C debugger.
  - In the file `c_cpp_properties.json` we can configure the compiler to use for linting purposes (intellisense).
    - It isn't strictly necessary to create this configuration file to be able to run and debug C programs in VSCode.
- VSCode workspace configuration files (`.json`) are stored in the subfolder `.vscode`.
- Run the cell below to create the folder `.vscode`.

**Note**

- This notebook doesn't dscribe the contents of these three files in detail. To learn more, visit: 
  - [task.json](https://code.visualstudio.com/docs/debugtest/tasks)
  - [launch.json](https://code.visualstudio.com/docs/debugtest/debugging)
  - [c_cpp_properties.json](https://code.visualstudio.com/docs/cpp/configure-intellisense)

In [6]:
import os
os.makedirs(".vscode", exist_ok=True)

---
## 1.6 Create the File `tasks.json`

- Run the cell below to create the file `tasks.json` in subfolder `.vscode`.

In [7]:
original = f'"HelloWorld"'
to_insert = "INSERT"

# Insert 'to_insert' before the second last character of 'original'
result = original[:-1] + r"\\" + to_insert + original[-1:]

print(result)


"HelloWorld\\INSERT"


In [8]:
import os, json

src_path = "${workspaceFolder}/src/*.c"
include_path = "${workspaceFolder}/include"
bin_path = "${workspaceFolder}/bin/main.exe"
if os_name == "windows":
    src_path = "${workspaceFolder}\\src\\*.c"
    include_path = "${workspaceFolder}\\include"
    bin_path = "${workspaceFolder}\\bin\\main.exe"

makedir_command = "mkdir"
makedir_args = ["-p", "src", "include", "bin"]
if os_name == "windows":
    makedir_command = windows_shell_path
    if windows_shell_name == "powershell.exe":
        makedir_args = ["-NoProfile", "-ExecutionPolicy", "Bypass", "-Command", "New-Item -ItemType Directory -Path 'src','include','bin' -Force -ErrorAction SilentlyContinue"]
    else:
        makedir_args = ["/c", "if not exist src mkdir src & if not exist include mkdir include & if not exist bin mkdir bin"]

clean_command = "find"
clean_args = ["./bin", "-type", "f", "-name", "*.exe", "-delete"]
if os_name == "windows":
    clean_command = windows_shell_path
    if windows_shell_name == "powershell.exe":
        clean_args = ["-NoProfile", "-ExecutionPolicy", "Bypass", "-Command", "Get-ChildItem -Path .\\bin -Include *.exe, *.ilk, *.pdb, *.obj -Recurse | Remove-Item -Force"]
    else:
        clean_args = ["/c", "del /s /q /f .\\bin\\*.exe 2>nul .\\bin\\*.ilk 2>nul .\\bin\\*.pdb 2>nul .\\bin\\*.obj pdb 2>nul"]

c_build_command = c_compiler_path
c_build_multi_args = ["-std=c17", "-Wall", "-g", src_path, "-I", include_path, "-o", bin_path] 
c_build_active_args = ["-std=c17", "-Wall", "-g", "${file}", "-o", bin_path]
if os_name == "windows" and c_compiler_name == "cl.exe":
    c_build_multi_args = ["/std:c17", "/nologo", "/Zi", "/EHsc", "/Fe:bin\\main.exe", "/Fo:bin\\", "/Fd:bin\\", "src\\*.c", "/I", "include"]
    c_build_active_args = ["/std:c17", "/nologo", "/Zi", "/EHsc", "/Fe:bin\\main.exe", "/Fo:bin\\", "/Fd:bin\\", "${file}"]

c_build_multi_args_opencl = ["-std=c17", "-Wall", "-g", src_path, "-I", include_path, "-I", opencl_include_dir, "-L", opencl_lib_dir, "-l", "OpenCL", "-o", bin_path] 
c_build_active_args_opencl = ["-std=c17", "-Wall", "-g", "${file}", "-I", opencl_include_dir, "-L", opencl_lib_dir, "-l", "OpenCL", "-o", bin_path]
if os_name == "windows" and c_compiler_name == "cl.exe":
    opencl_lib_file = opencl_lib_dir[:-1] + r"\OpenCL.lib" + opencl_lib_dir[-1:]
    c_build_multi_args_opencl = ["/std:c17", "/nologo", "/Zi", "/EHsc", "/Fe:bin\\main.exe", "/Fo:bin\\", "/Fd:bin\\", "src\\*.c", "/I", "include", "/I", opencl_include_dir, "/link", opencl_lib_file]
    c_build_active_args_opencl = ["/std:c17", "/nologo", "/Zi", "/EHsc", "/Fe:bin\\main.exe", "/Fo:bin\\", "/Fd:bin\\", "${file}", "/I", opencl_include_dir, "/link",  opencl_lib_file]
if os_name == "osx" and c_compiler_name == "clang":
    c_build_multi_args_opencl = ["-std=c17", "-Wall", "-g", src_path, "-I", include_path, "-framework", "OpenCL", "-o", bin_path] 
    c_build_active_args_opencl = ["-std=c17", "-Wall", "-g", "${file}", "-framework", "OpenCL", "-o", bin_path]

tasks_json = {
    "version": "2.0.0",
    "tasks": [
        {
            "type": "shell",
            "label": "Make directories",
            "command": makedir_command,
            "args": makedir_args,
            "problemMatcher": []
        },
        {
            "type": "shell",
            "label": "Clean .exe files",
            "dependsOn": ["Make directories"],
            "command": clean_command,
            "args": clean_args,
            "problemMatcher": []
        },
        {
            "type": "shell",
            "label": "opencl: build multi file",
            "dependsOn": ["Clean .exe files"],
            "command": c_build_command,
            "args": c_build_multi_args_opencl,
            "options": {
                "cwd": "${workspaceFolder}"
            },
            "problemMatcher": [
                "$gcc"
            ],
            "group": {
                "kind": "build",
                "isDefault": False
            },
            "detail": f"compiler: {c_compiler_path}"
        },
        {
            "type": "shell",
            "label": "opencl: build active file",
            "dependsOn": ["Clean .exe files"],
            "command": c_build_command,
            "args": c_build_active_args_opencl,
            "options": {
                "cwd": "${fileDirname}"
            },
            "problemMatcher": [
                "$gcc"
            ],
            "group": {
                "kind": "build",
                "isDefault": True
            },
            "detail": f"compiler: {c_compiler_path}"
        }
    ]
}

os.makedirs(".vscode", exist_ok=True)
json_string = json.dumps(tasks_json, indent=4)
with open(".vscode/tasks.json", "w") as f:
    json.dump(tasks_json, f, indent=4)

---
## 1.7 Create the File `launch.json`

- Run the cell below to create the file `launch.json` in subfolder `.vscode`.

In [9]:
import os, json

if "gdb" in c_debugger_name:
    c_debugger_type = "cppdbg"
    c_debugger_mi_mode = "gdb"
    stop_at_entry_name = "stopAtEntry"
    environment = True
    console = False
    setupcommands = [{"description": "Enable pretty-printing for gdb", "text": "-enable-pretty-printing", "ignoreFailures": True}]
if "lldb" in c_debugger_name:
    c_debugger_type = "lldb"
    c_debugger_mi_mode = "lldb"
    stop_at_entry_name = "stopOnEntry"
    environment = False
    console = False
    setupcommands = None
if c_debugger_name == "cdb.exe":
    c_debugger_name = "msvc"
    c_debugger_path = None
    c_debugger_type = "cppvsdbg"
    c_debugger_mi_mode = None
    stop_at_entry_name = "stopAtEntry"
    environment = True
    console = True
    setupcommands = None

launch_json = {
    "version": "0.2.0",
    "configurations": [
        {
            "name": "opencl: launch multi file",
            "preLaunchTask": "opencl: build multi file",
            "type": c_debugger_type,
            "request": "launch",
            "program": bin_path,
            "args": [],
            f"{stop_at_entry_name}": False,
            "cwd": "${workspaceFolder}"
        },
        {
            "name": "opencl: launch active file",
            "preLaunchTask": "opencl: build active file",
            "type": c_debugger_type,
            "request": "launch",
            "program": bin_path,
            "args": [],
            f"{stop_at_entry_name}": False,
            "cwd": "${workspaceFolder}"
        }
    ]
}

if environment:
    for i in range(2):
        launch_json["configurations"][i]["environment"] = []
if c_debugger_name != "lldb":
    if console:
        for i in range(2):
            launch_json["configurations"][i]["console"] = "integratedTerminal" # "externalTerminal"
    else:
        for i in range(2):
            launch_json["configurations"][i]["externalConsole"] = False # True
    if c_debugger_mi_mode:
        for i in range(2):
            launch_json["configurations"][i]["MIMode"] = c_debugger_mi_mode
    if c_debugger_path:
        for i in range(2):
            launch_json["configurations"][i]["miDebuggerPath"] = c_debugger_path
    if setupcommands:
        for i in range(2):
            launch_json["configurations"][i]["setupCommands"] = setupcommands


os.makedirs(".vscode", exist_ok=True)
with open(".vscode/launch.json", "w") as f:
    json.dump(launch_json, f, indent=4)

---
## 1.8 Create the File `c_cpp_properties.json`

- Run the cell below to create the file `c_cpp_properties.json` in subfolder `.vscode`.

In [10]:
import os, json

if os_name == "linux" and c_compiler_name == "gcc":
    intelliSenseMode = "linux-gcc-x64"
    # intelliSenseMode = "linux-gcc-arm64"
if os_name == "linux" and c_compiler_name == "clang":
    intelliSenseMode = "linux-clang-x64"
    # intelliSenseMode = "linux-clang-arm64"
if os_name == "osx" and c_compiler_name == "gcc":
    intelliSenseMode = "macos-gcc-x64"
    # intelliSenseMode = "macos-gcc-arm64"
if os_name == "osx" and c_compiler_name == "clang":
    intelliSenseMode = "macos-clang-x64"
    # intelliSenseMode = "macos-clang-arm64"
if os_name == "windows" and c_compiler_name == "gcc.exe":
    intelliSenseMode = "windows-gcc-x64"
if os_name == "windows" and c_compiler_name == "clang.exe":
    intelliSenseMode = "windows-clang-x64"
if os_name == "windows" and c_compiler_name == "cl.exe":
    intelliSenseMode = "windows-msvc-x64"

launch_json = {
    "configurations": [
        {
            "name": "Linter",
            "includePath": [
                "${workspaceFolder}/**"
            ],
            "defines": [],
            "compilerPath": c_compiler_path,
            "cStandard": "c17",
            "cppStandard": "c++17",
            "intelliSenseMode": intelliSenseMode
        }
    ],
    "version": 4
}

os.makedirs(".vscode", exist_ok=True)
with open(".vscode/c_cpp_properties.json", "w") as f:
    json.dump(launch_json, f, indent=4)

---
## 1.9 VSCode Extensions

**CoLab**
- SKip to [1.10 Using Built-in Cell Magic `%%writefile`](#110-using-built-in-cell-magic-writefile).

**Linux/Mac/Windows**

- To develop OpenCL programs in C with VSCode, we need a few VSCode extensions (the last two are only needed for Jupyter Notebooks).
  - C/C++ Extension Pack: https://marketplace.visualstudio.com/items?itemName=ms-vscode.cpptools-extension-pack
  - CodeLLDB: https://marketplace.visualstudio.com/items?itemName=vadimcn.vscode-lldb
  - Makefile Tools: https://marketplace.visualstudio.com/items?itemName=ms-vscode.makefile-tools
  - Jupyter: https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter
  - Python: https://marketplace.visualstudio.com/items?itemName=ms-python.python

- Run the cell below to install any missing VSCode extensions.

In [11]:
!code --install-extension ms-vscode.cpptools-extension-pack --force
!code --install-extension vadimcn.vscode-lldb --force
!code --install-extension ms-vscode.makefile-tools --force
!code --install-extension ms-toolsai.jupyter --force
!code --install-extension ms-python.python --force

Installing extensions...
Extension 'ms-vscode.cpptools-extension-pack' is already installed.
Installing extensions...
Extension 'vadimcn.vscode-lldb' is already installed.
Installing extensions...
Extension 'ms-vscode.makefile-tools' is already installed.
Installing extensions...
Extension 'ms-toolsai.jupyter' is already installed.
Installing extensions...
Extension 'ms-python.python' is already installed.


---
## 1.10 Using Built-in Cell Magic `%%writefile`

- The cell magic `%%writefile filename`, writes the contents of a notebook cell to the specified `filename` (or file path).
  - This functionality is built-in to Jupyter Notebooks (it's not an extension).
  - We can use it to write any code cell contents to a file in the file system.
  - Let's write some OpenCL code to the file `main.c` with a kernel in `kernel.c`.
- Run the two cells below.
- Then inspect the resulting files `kernel.cl` and `main.c`, which contain each code cell's contents (except for the cell magic command `%%writefile filename`).

In [12]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>
#define CL_TARGET_OPENCL_VERSION 300
#include <CL/cl.h>

const char *source =
"__kernel void mykernel()\n"
"{\n"
"    printf(\"Hello World!\\n\");\n"
"}\n";

int main()
{
    cl_int err;

    cl_platform_id platform;
    cl_uint num_platforms;
    err = clGetPlatformIDs(1, &platform, &num_platforms);

    cl_device_id device;
    cl_uint num_devices;
    err = clGetDeviceIDs(platform, CL_DEVICE_TYPE_DEFAULT, 1, &device, &num_devices);

    cl_context context = clCreateContext(NULL, 1, &device, NULL, NULL, &err);

    cl_command_queue queue = clCreateCommandQueueWithProperties(context, device, 0, &err);

    cl_program program = clCreateProgramWithSource(context, 1, &source, NULL, &err);
    err = clBuildProgram(program, 1, &device, NULL, NULL, NULL);

    cl_kernel kernel = clCreateKernel(program, "mykernel", &err);

    size_t global_size = 1;
    size_t local_size = 1;
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);

    clFinish(queue);

    clReleaseKernel(kernel);
    clReleaseProgram(program);
    clReleaseCommandQueue(queue);
    clReleaseContext(context);

    return 0;
}

Writing main.c


---
## 1.11 Compiling and Executing an OpenCL Program From a Notebook Code Cell

- We can compile and execute an OpenCL program from a notebook code cell using the syntax `!<shell command>`, where:
  - `!` indicates that the succeeding text on the same row should be sent to the shell (terminal).
  - `<shell command>` is the shell (terminal) command we want to execute.
  - Standard output is redirected to the cell output.

- Run the cell below to see what the build command and execute command is in your shell:
  - The *build single file* command compiles and links the file `main.c` in your workspace folder and places the executable file `main.exe` in the `bin` folder.
  - The *build multi file* command compiles and links all `.c` files in the `src` and `.h` files in the `include` folder and places the executable file `main.exe` in the `bin` folder.
  - The *execute* command executes the file `main.exe` in the `bin` folder.

In [13]:
build_single_file_command = [f'"{c_build_command}"'] + c_build_active_args_opencl
build_single_file_command = " ".join(build_single_file_command).replace('${file}', 'main.c').replace('${workspaceFolder}', '.')

build_multi_file_command = [f'"{c_build_command}"'] + c_build_multi_args_opencl
build_multi_file_command = " ".join(build_multi_file_command).replace('${file}', 'main.c').replace('${workspaceFolder}', '.')

execute_command = bin_path.replace('${workspaceFolder}', '.')

print(f'Build single file command : {build_single_file_command}')
print(f'Build multi file command  : {build_multi_file_command}')
print(f'Execute command           : {execute_command}')

Build single file command : "/usr/bin/gcc" -std=c17 -Wall -g main.c -I "/usr/local/cuda-12.8/targets/x86_64-linux/include" -L "/usr/local/cuda-12.8/targets/x86_64-linux/lib" -l OpenCL -o ./bin/main.exe
Build multi file command  : "/usr/bin/gcc" -std=c17 -Wall -g ./src/*.c -I ./include -I "/usr/local/cuda-12.8/targets/x86_64-linux/include" -L "/usr/local/cuda-12.8/targets/x86_64-linux/lib" -l OpenCL -o ./bin/main.exe
Execute command           : ./bin/main.exe


- Run the cell below to:
  - Create the folders `bin`, `src`, and `include` in your workspace folder if they don't already exist.
  - Build the single source code file `main.c` in your workspace folder into the executable file `main.exe` in the `bin` folder.
  - Run the executable file `main.exe` in the `bin` folder.
- Notice the file `main.exe` has been created in the file system (in the `bin` folder), and the program's output is shown as the cell's output in the notebook. 

In [ ]:
import os
os.makedirs("src", exist_ok=True)
os.makedirs("include", exist_ok=True)
os.makedirs("bin", exist_ok=True)

!{build_single_file_command}
!{execute_command}

Hello World!


---
## 1.12 Compiling and Debugging a Single-file OpenCL Program

Let's see `tasks.json` and `launch.json` in action for a single-file (`.c`) OpenCL program.

**CoLab**
- Skip to [2. OpenCL Basics](#2-opencl-basics).

**Linux/Windows/Mac**
- First, let's create the file `main.c` in the cell below.

In [15]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>
#define CL_TARGET_OPENCL_VERSION 300
#include <CL/cl.h>

const char *source =
"__kernel void mykernel()\n"
"{\n"
"    printf(\"Hello World!\\n\");\n"
"}\n";

void checkOpenCL(cl_int err, const char *msg)
{
    if (err != CL_SUCCESS)
    {
        fprintf(stderr, "%s failed: %d\n", msg, err);
        exit(EXIT_FAILURE);
    }
}

int main()
{
    cl_int err;

    // 1. Platform
    cl_platform_id platform;
    cl_uint num_platforms;
    err = clGetPlatformIDs(1, &platform, &num_platforms);

    // 2. Device
    cl_device_id device;
    cl_uint num_devices;
    err = clGetDeviceIDs(platform, CL_DEVICE_TYPE_DEFAULT, 1, &device, &num_devices);

    // 3. Context
    cl_context context = clCreateContext(NULL, 1, &device, NULL, NULL, &err);

    // 4. Command queue
    cl_command_queue queue = clCreateCommandQueueWithProperties(context, device, 0, &err);

    // 5. Program
    cl_program program = clCreateProgramWithSource(context, 1, &source, NULL, &err);
    checkOpenCL(err, "clCreateProgramWithSource");
    err = clBuildProgram(program, 1, &device, NULL, NULL, NULL);

    // (Optional) Check build log if needed
    if (err != CL_SUCCESS)
    {
        char log[2048];
        clGetProgramBuildInfo(program, device, CL_PROGRAM_BUILD_LOG, sizeof(log), log, NULL);
        printf("Build log:\n%s\n", log);
        exit(EXIT_FAILURE);
    }

    // 6. Kernel
    cl_kernel kernel = clCreateKernel(program, "mykernel", &err);
    checkOpenCL(err, "clCreateKernel");

    // 7. Launch
    size_t global_size = 1;
    size_t local_size = 1;
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);

    // 8. Wait and finish
    clFinish(queue);

    // 9. Cleanup
    clReleaseKernel(kernel);
    clReleaseProgram(program);
    clReleaseCommandQueue(queue);
    clReleaseContext(context);

    return 0;
}

Overwriting main.c


- Now, let's debug the file `main.c`.
  - Open the file `main.c` in VSCode's editor.
  - Set a breakpoint on the two `return` statements (`F9`).
  - Switch to the `Run and Debug` view (Linux/Windows: `Ctrl + Shift + D`, Mac: `Cmd + Shift + D`).
  - In the drop-down combobox, select the launch configuration `<COMPILER>: launch active file`, where `<COMPILER>`is the name of your C compiler.
  - Click the green `Play` icon.
  - Use the debug toolbar in the top-middle of VSCode to debug the code.
    - Notice the debugger stops at the breakpoints.
      - This is because we are using a C debugger compatible with your chosen C compiler.
    - Notice you can view variables (local, registers), watch variables, view the call stack, and toggle breakpoints in the `Run and Debug` view.
  - Stop debugging (red `Square` icon in the debug toolbar).
- Next, look at the status bar (at the bottom of VSCode) where you will see the name of the launch configuration `<COMPILER>: launch active file`.
  - Click on it, and select `<COMPILER>: launch active file` again (make sure `main.c` is the active file in the editor, not the notebook).
    - This is an alternative method to start a debug session.
  - Stop debugging.
- Press `F5` (make sure `main.c` is active in VSCode's editor), which is a third alternative to launch the debugger.
  - This launches the debug configuration with `preLaunchTask` set to the default task (in `tasks.json`).
  - Stop debugging.
- Press (Linux/Windows: `Ctrl + Shift + B`, Mac: `Ctrl + Shift + B`) to execute the default build task (in `tasks.json`).
  - Make sure `main.c` is active in VSCode's editor (since the default build task is set to the active file task).
  - Notice the compiled executable `main.exe` is placed in the subfolder `bin` (configured in the default build task).
    - This is also where the debugger finds the executable `main.exe` (configured in `launch.json`).

- Remeber, you can always compile a single `main.c` file in your workspace folder and run it using the commands below.

In [16]:
!{build_single_file_command}
!{execute_command}

Hello World!


---
## 1.13 Compiling and Debugging a Multi-file OpenCL Program

**CoLab**
- Skip to [2. OpenCL Basics](#2-opencl-basics).

**Linux/Windows/Mac**
- Let's see `tasks.json` and `launch.json` in action for a multi-file (`*.c`) OpenCL program.
  - First We will create two source code files `.c` in the `src` folder, and one header file `.h` in the `include` folder.
    - We will use the same code as before, but will place the OpenCL kernel (function) code in its own `.cl` file, together with a source code file `.c` for loading it and its prototype in a header file `.h`.
  - Then we will use:
    - The other (non-default) build task in `tasks.json` to build the executable.
    - The other launch configuration (linked to the non-default build task) in `launch.json` to debug it.

- Run the two cells below to create:
  - The kernel source code file `kernel.cl` in the folder `src`.
  - The main source code file `main.c` in the folder `src`.

In [18]:
%%writefile src/kernel.cl
__kernel void mykernel()
{
    printf("Hello World!\n");
}

Writing src/kernel.cl


In [19]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#define CL_TARGET_OPENCL_VERSION 300
#include <CL/cl.h>

#define KERNEL_FILE "src/kernel.cl"

char* read_kernel_source(const char* filename)
{
    FILE* fp = fopen(filename, "r");
    if (!fp)
    {
        fprintf(stderr, "Failed to open kernel file: %s\n", filename);
        exit(1);
    }
    fseek(fp, 0, SEEK_END);
    long size = ftell(fp);
    rewind(fp);
    char* src = (char*)malloc(size + 1);
    fread(src, 1, size, fp);
    src[size] = '\0';
    fclose(fp);
    return src;
}

int main()
{
    cl_int err;

    // 1. Platform
    cl_platform_id platform;
    cl_uint num_platforms;
    err = clGetPlatformIDs(1, &platform, &num_platforms);

    // 2. Device
    cl_device_id device;
    cl_uint num_devices;
    err = clGetDeviceIDs(platform, CL_DEVICE_TYPE_DEFAULT, 1, &device, &num_devices);

    // 3. Context
    cl_context context = clCreateContext(NULL, 1, &device, NULL, NULL, &err);

    // 4. Command queue
    cl_command_queue queue = clCreateCommandQueueWithProperties(context, device, 0, &err);

    // 5. Program
    char* source = read_kernel_source(KERNEL_FILE); // Read kernel source
    cl_program program = clCreateProgramWithSource(context, 1, (const char**)&source, NULL, &err);
    err = clBuildProgram(program, 1, &device, NULL, NULL, NULL);

    // (Optional) Check build log if needed
    if (err != CL_SUCCESS)
    {
        char log[2048];
        clGetProgramBuildInfo(program, device, CL_PROGRAM_BUILD_LOG, sizeof(log), log, NULL);
        printf("Build log:\n%s\n", log);
        exit(EXIT_FAILURE);
    }

    // 6. Kernel
    cl_kernel kernel = clCreateKernel(program, "mykernel", &err);

    // 7. Launch
    size_t global_size = 1;
    size_t local_size = 1;
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);

    // 8. Wait and finish
    clFinish(queue);

    // 9. Cleanup
    free(source);
    clReleaseKernel(kernel);
    clReleaseProgram(program);
    clReleaseCommandQueue(queue);
    clReleaseContext(context);

    return 0;
}

Writing src/main.c


- Build the multi-file C program.
  - Notice the multi-file build task in `tasks.json` isn't the default build task (`isDefault` is not set to `true` under `group`).
  - Therefore, we can't use (Linux/Windows: `Ctrl + Shift + B`, mac: `Cmd + Shift + B`).
  - Instead we can:
    - Bring up the Command Palette (Linux/Windows: `Ctrl + Shift + P`, mac: `Cmd + Shift + P`).
    - Choose `Tasks: Run Task` and select the task `<COMPILER>: build multi file`, where `<COMPILER>` is the name of your chosen C compiler.
  - The executable `main.exe` is placed in the `bin` folder (as configured in `tasks.json`).
- Debug the multi-file C program.
  - Open `main.c` and `math_utils.c` in the `src` folder and set breakpoints on the two `return` statements.
  - Notice the multi-file launch task in `launch.json` isn't linked to the default build task (in `tasks.json`).
    - Therefore, we can't use `F5`.
    - Instead we can:
      - Switch the the `Run and Debug` view, select `<COMPILER>: launch multi file` from the drop-down list, and click the green `Play` icon.
      - Or select `<COMPILER>: launch multi file` from the status bar (at the bottom of VSCode).
    - The C program is built and the debugger lauched, attaching to the executable `main.exe` in the `bin` folder.
  - Stop debugging.

- Remeber, you can always compile a multi-file C program (`src/*.c`, `include/*.h`) and run it using the commands below.

In [20]:
!{build_multi_file_command}
!{execute_command}

Hello World!


---
# 2. OpenCL Basics
---

- Now we know how to create OpenCL programs in C (single-file, multi-file, and in a Jupyter Notebook cell).
- Going forward, we will explore fundamental OpenCL programming concepts as single-file programs in notebook cells using the `%%writefile` cell magic command.
- The Khronos Group's documentation is a good source to learn more about OpenCL:
    - [Khronos Group OpenCL](https://www.khronos.org/opencl)

---
## 2.1 Listing OpenCL-enabled Devices and Properties

First, let's find out what OpenCL-enabled devices are available on your computer.

### Using `clinfo`

- The simplest way to list OpenCL-enabled devices is using the tool `clinfo`.
  - The top row shows you the number of platforms on your system:
    - Example platforms: Nvidia, AMD, Intel, etc.
  - For each platform, it shows you which devices are available:
    - Example devices: an Nvidia GPU, an AMD CPU, an AMD GPU, an Intel CPU, etc.
  - For each device, it shows you information about that device.
- Run the cell below to see the OpenCL-enabled devices on your system.

In [21]:
!clinfo

Number of platforms                               1
  Platform Name                                   NVIDIA CUDA
  Platform Vendor                                 NVIDIA Corporation
  Platform Version                                OpenCL 3.0 CUDA 12.8.90
  Platform Profile                                FULL_PROFILE
  Platform Extensions                             cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_khr_fp64 cl_khr_3d_image_writes cl_khr_byte_addressable_store cl_khr_icd cl_khr_gl_sharing cl_nv_compiler_options cl_nv_device_attribute_query cl_nv_pragma_unroll cl_nv_copy_opts cl_khr_gl_event cl_nv_create_buffer cl_khr_int64_base_atomics cl_khr_int64_extended_atomics cl_nv_kernel_attribute cl_khr_device_uuid cl_khr_pci_bus_info cl_khr_external_semaphore cl_khr_external_memory cl_khr_external_semaphore_opaque_fd cl_khr_external_memory_opaque_fd cl_khr_semaphore
  Platform Extensions

### Using C Code

- We can also find out what Platforms and OpenCL-enabled devices are available using C code.
  - The code below lists important properties that will become familiar the more you learn about OpenCL (for optimization purposes).
- Run the cell below to list platforms, OpenCL-enabled devices, and (some of) their properties.

In [24]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>

#define CL_TARGET_OPENCL_VERSION 300
#include <CL/cl.h>

void checkOpenCL(cl_int err, const char *msg)
{
    if (err != CL_SUCCESS)
    {
        fprintf(stderr, "%s failed: %d\n", msg, err);
        exit(EXIT_FAILURE);
    }
}

int main()
{
    cl_uint numPlatforms;
    cl_int err;

    err = clGetPlatformIDs(0, NULL, &numPlatforms);
    checkOpenCL(err, "clGetPlatformIDs (count)");

    if (numPlatforms == 0)
    {
        puts("No OpenCL platforms found.");
        return 1;
    }

    cl_platform_id *platforms = (cl_platform_id *) malloc(sizeof(cl_platform_id) * numPlatforms);
    err = clGetPlatformIDs(numPlatforms, platforms, NULL);
    checkOpenCL(err, "clGetPlatformIDs");

    for (cl_uint p = 0; p < numPlatforms; ++p)
    {
        char platformName[128];
        clGetPlatformInfo(platforms[p], CL_PLATFORM_NAME, sizeof(platformName), platformName, NULL);
        printf("Platform %u: %s\n", p, platformName);

        cl_uint numDevices = 0;
        err = clGetDeviceIDs(platforms[p], CL_DEVICE_TYPE_ALL, 0, NULL, &numDevices);
        if (err != CL_SUCCESS || numDevices == 0)
        {
            puts("  No devices found.");
            continue;
        }

        cl_device_id *devices = (cl_device_id *) malloc(sizeof(cl_device_id) * numDevices);
        err = clGetDeviceIDs(platforms[p], CL_DEVICE_TYPE_ALL, numDevices, devices, NULL);
        checkOpenCL(err, "clGetDeviceIDs");

        for (cl_uint i = 0; i < numDevices; ++i)
        {
            char name[128];
            cl_uint compute_units, max_work_group_size, clock_frequency;
            cl_ulong global_mem, local_mem, constant_mem;
            cl_device_type type;

            clGetDeviceInfo(devices[i], CL_DEVICE_NAME, sizeof(name), name, NULL);
            clGetDeviceInfo(devices[i], CL_DEVICE_TYPE, sizeof(type), &type, NULL);
            clGetDeviceInfo(devices[i], CL_DEVICE_MAX_COMPUTE_UNITS, sizeof(compute_units), &compute_units, NULL);
            clGetDeviceInfo(devices[i], CL_DEVICE_MAX_WORK_GROUP_SIZE, sizeof(max_work_group_size), &max_work_group_size, NULL);
            clGetDeviceInfo(devices[i], CL_DEVICE_MAX_CLOCK_FREQUENCY, sizeof(clock_frequency), &clock_frequency, NULL);
            clGetDeviceInfo(devices[i], CL_DEVICE_GLOBAL_MEM_SIZE, sizeof(global_mem), &global_mem, NULL);
            clGetDeviceInfo(devices[i], CL_DEVICE_LOCAL_MEM_SIZE, sizeof(local_mem), &local_mem, NULL);
            clGetDeviceInfo(devices[i], CL_DEVICE_MAX_CONSTANT_BUFFER_SIZE, sizeof(constant_mem), &constant_mem, NULL);

            printf("  - Device %u: %s\n", i, name);
            printf("    - %-31s : %s\n", "Type",
                (type == CL_DEVICE_TYPE_GPU) ? "GPU" :
                (type == CL_DEVICE_TYPE_CPU) ? "CPU" :
                (type == CL_DEVICE_TYPE_ACCELERATOR) ? "Accelerator" : "Other");
            printf("    - %-31s : %u\n", "Compute Units (CU)", compute_units);
            printf("    - %-31s : %u\n", "Max Work Group Size", max_work_group_size);
            printf("    - %-31s : %.2f MHz\n", "Clock Frequency", clock_frequency * 1.0);
            printf("    - %-31s : %llu bytes\n", "Global Memory Size", (long long unsigned int)global_mem);
            printf("    - %-31s : %llu bytes\n", "Local Memory Size (shared)", (long long unsigned int)local_mem);
            printf("    - %-31s : %llu bytes\n", "Constant Memory Size", (long long unsigned int)constant_mem);
            printf("\n");
        }

        free(devices);
    }

    free(platforms);
    return 0;
}

Overwriting main.c


In [25]:
!{build_single_file_command}
!{execute_command}

Platform 0: NVIDIA CUDA
  - Device 0: NVIDIA RTX 2000 Ada Generation Laptop GPU
    - Type                            : GPU
    - Compute Units (CU)              : 24
    - Max Work Group Size             : 32766
    - Clock Frequency                 : 1455.00 MHz
    - Global Memory Size              : 8198619136 bytes
    - Local Memory Size (shared)      : 49152 bytes
    - Constant Memory Size            : 65536 bytes



---
## 2.2 Hello World in Host Code (CPU)

- The program in the cell below is a simple C program (no OpenCL code) that runs on the host (CPU).
  - We include the necessary header files:
    - `stdio.h` for `printf`.

      ```c
      #include <stdio.h>
      ```
  - Then we define the `main()` function:
    - We print out the text `Hello World!`.
    - Then we return the exit code `0` to the operating system.
    
      ```c
      int main(void)
      {
        printf("Hello World!\n");
        
        return 0;
      }
      ```
- Run the cell below to see the output.

In [26]:
%%writefile main.c
#include <stdio.h>

// Host entry point (a normal C main function)
int main(void)
{
   printf("Hello World!\n");
   
   return 0;
}

Overwriting main.c


In [27]:
!{build_single_file_command}
!{execute_command}

Hello World!


---
## 2.3 Hello World in Device Code (GPU)

- The program in the cell below is a simple OpenCL program that runs code on the host (CPU) and the device (GPU).
  - We include the necessary header files:
    - `stdio.h` for `printf` and `fprintf`.
    - `stdlib.h` for the function `exit` and the symbolic constant `EXIT_FAILURE`.
    - `CL/cl.h` for all the OpenCL-related functions, type, symbolic constants, etc. (most of them have a name that starts with) `cl`.
      - We also preceed this `#include`  with a `#define` which chooses the OpenCL version we want to use

      ```c
      #include <stdio.h>
      #include <stdlib.h>
      #define CL_TARGET_OPENCL_VERSION 300 // the version of OpenCL we want to use (must be placed above the include below)
      #include <CL/cl.h>                   // contains OpenCL function prototypes, types, symbolic constants, etc.
      ```
  - Then we define an OpenCL kernel function:
    - `mykernel()` is the name of the **kernel function** (it can be any name we like, but we will stick with `mykernel` in this notebook).
    - A **kernel function** runs on the device (GPU) and is **launched** (called) from the host (CPU).
    - `__kernel` is an OpenCL **qualifier** (qualifying a function with `__kernel` makes the function a **kernel function**.
    - The kernel function takes no arguments (`void`), prints the text `Hello World!`, and does NOT return a value (`void`).
    - All **kernel functions** MUST have a `void` return type.
    - Notice this is defined as a C litteral string, where each "line" within the string needs a newline character `\n`.

      ```c
      const char *source =
      "__kernel void mykernel()\n"
      "{\n"
      "    printf(\"Hello World!\\n\");\n"
      "}\n";
      ```
  - Following this, we define two helper functions `checkOpenCL` and `checkOpenCLBuildLog`:
    - `checkOpenCL` takes an OpenCL integer `cl_int` (containing an OpenCL error code `err`) and a C string (a message `msg`) as arguments.
      - It then checks if the error code represents an error (`CL_SUCCESS` means no error), and if so:
        - prints out the error code together with the message
        - exits the program
      
      ```c
      void checkOpenCL(cl_int err, const char *msg)
      {
          if (err != CL_SUCCESS)
          {
              fprintf(stderr, "%s failed: %d\n", msg, err);
              exit(EXIT_FAILURE);
          }
      }
      ```
    - `checkOpenCLBuildLog` takes an OpenCL error code of type `cl_int`, OpenCL program of type `cl_program`, and an OpenCL device of type `cl_device`.
      - It then checks if the error code represents an error (`CL_SUCCESS` means no error), and if so:
        - declares a buffer `log` and calls the OpenCL function `clGetProgramBuildInfo` to extract build information from the program into the buffer.
          - OpenCL kernels (programs) are compiled as part of the C program, so to find out if a compilation error occured, we use this construct.
        - prints out the build log in the buffer (this is how we can view errors from compiling our OpenCL kernels).
        - exits the program
      
      ```c
      void checkOpenCLBuildLog(cl_int err, cl_program program, cl_device_id device)
      {
          if (err != CL_SUCCESS)
          {
              char log[2048];
              clGetProgramBuildInfo(program, device, CL_PROGRAM_BUILD_LOG, sizeof(log), log, NULL);
              printf("Build log:\n%s\n", log);
              exit(EXIT_FAILURE);
          }
      }
      ```
  - Lastly we define the `main()` function:
    - First, we declare a variable `err` of type `cl_int`, in which we can store error codes returned by OpenCL functions.
      - Almost all OpenCL functions return an error, or populate a `cl_int` parameter with an error code.
      - This is the error code we pass as an argument to the help functions above.
    
      ```c
      cl_int err;
      ```
    - Then we find out what OpenCL platforms are available on our system.
      - The variable `platform` of type `cl_platform_id` will contain a platform ID (e.g. for an Nvidia, AMD, or Intel platform).
      - The variable `num_platforms` of type `cl_uint` will contain the number of platforms on our system.
      - The call to the OpenCL function `clGetPlatformIDs` populates the two variables above with values.
        - The first argument `1` to the function, means we only want `1` platform ID stored in the `platform` variable (the default platform on our system).
        - Notice the function `clGetPlatformIDs` returns an error code which we store in the variable `err` we declared above.
      - Finally, we call our helper function `checkOpenCL` passing in the error code `err`, and a message `"clGetPlatformIDs"` (i.e. the name of the OpenCL function).
        - Our helper function will check so see if the error code constitutes an error, and if so, print it out with the message, then exit the C program.
      
      ```c
      // 1. Platform
      cl_platform_id platform;
      cl_uint num_platforms;
      err = clGetPlatformIDs(1, &platform, &num_platforms);
      checkOpenCL(err, "clGetPlatformIDs");
      ```
    - The next step is to find out what OpenCL-enabled devices are available for the platform we chose above.
      - The variable `device` of type `cl_device_id` will contain a device ID (e.g. for an Nvidia GPU, AMD GPU/CPU, or Intel CPU, etc.).
      - The variable `num_devices` of type `cl_uint` will contain the number of devices for the chosen platform.
      - The call to the OpenCL function `clGetDeviceIDs` populates the two variables above with values.
        - The first argument `platform` to the function contains the platform ID we chose above.
        - The second argument `CL_DEVICE_TYPE_DEFAULT` means we only want the default device for this platform (e.g. an AMD plarfotm can contain both a GPU and a CPU).
        - The third argument `1` means we only want `1` device ID (since we are using `CL_DEVICE_TYPE_DEFAULT`).
        - The final two arguments are the variables that will be populated with values.
      - Finally, we call our helper function `checkOpenCL` passing in the returned error code `err`.
      
      ```c
      // 2. Device
      cl_device_id device;
      cl_uint num_devices;
      err = clGetDeviceIDs(platform, CL_DEVICE_TYPE_DEFAULT, 1, &device, &num_devices);
      checkOpenCL(err, "clGetDeviceIDs");
      ```
    - Great, so now we have chosen a platform, and the device within the platform we want to use, so the next step is to create a `context`.
      - An OpenCL context is the abstraction used to manage a device, containing objects such as command queues, memory buffers, etc. described below.
      - The variable `context` of type `cl_context` will contain the context for our chosen device.
      - The call to the OpenCL function `clCreateContext` returns the `context`.
        - The function taks multiple arguments, but we are only interested in the second, third, and final argument in this notebook.
        - The second argument `1` is the number contexts we want.
        - The third argument `device` is our chosen device from above.
        - The final argument `err` is our variable for storing the error code produced by this function.
          - Here we see a second way an OpenCL function populates a variable with an error code (passed as an argument, since the function's return value is a context).
      - Finally, we call our helper function `checkOpenCL` passing in the returned error code `err`.
      
      ```c
      // 3. Context
      cl_context context = clCreateContext(NULL, 1, &device, NULL, NULL, &err);
      checkOpenCL(err, "clCreateContext");
      ```
    - Now that we have a context, we need to create a command queue in it.
      - An OpenCL command queue is used to queue commands to the chosen device within a context, such as allocating memory on the device, launching a kernel, etc.
      - The variable `queue` of type `cl_command_queue` will contain the command queue for our chosen device within our context.
      - The call to the OpenCL function `clCreateCommandQueueWithProperties` returns the command `queue`.
        - The first argument `context` is our context from above.
        - The second argument `device` is our device from above.
        - The third argument `0` (or `NULL`) means we are using default command queue properties (we can set additional properties, as we will see later).
        - The final argument `err` is our variable for storing the error code produced by this function.
      - Finally, we call our helper function `checkOpenCL` passing in the returned error code `err`.
      
      ```c
      // 4. Command queue
      cl_command_queue queue = clCreateCommandQueueWithProperties(context, device, 0, &err);
      checkOpenCL(err, "lCreateCommandQueueWithProperties");
      ```
    - Once we have a means of queueing commands to a device, we can create and build a program for our OpenCL kernel (from the source code for the kernel above).
      - The variable `program` of type `cl_program` will contain the program for our kernel once it has been compiled.
      - The call to the OpenCL function `clCreateProgramWithSource` returns the command `program`, where we're not interested in the fourth argument at the moment.
        - The first argument `context` is our context from above.
        - The second argument `1` means we only want to create one program (kernel).
        - The third argument `source` is the source code for our device kernel (from the source code for the kernel above).
        - The final argument `err` is our variable for storing the error code produced by this function.
      - We then call our helper function `checkOpenCL` passing in the error code `err`.
      - Now we need to compile our kernel'äs source code, which is done with the OpenLC function `clBuildProgram`.
        - We're only interested in the first three arguments in this notebook.
        - The first argument `program` is our program from above.
        - The second argument `1` means we only want compile the sourced code for one kernel.
        - The third argument `device` is our device from above (i.e. this kernel is for this device).
      - Finally, we call our helper function `checkOpenCL` passing in the returned error code `err`.
      
      ```c
      // 5. Program
      cl_program program = clCreateProgramWithSource(context, 1, &source, NULL, &err);
      checkOpenCL(err, "clCreateProgramWithSource");
      err = clBuildProgram(program, 1, &device, NULL, NULL, NULL);
      checkOpenCLBuildLog(err, program, device);
      ```
    - The next step is to create the device kernel we want to use.
      - The variable `kernel` of type `cl_kernel` will contain the compiled kernel.
      - The call to the OpenCL function `clCreateKernel` returns the command `kernel`.
        - The first argument `program` is our program from above.
        - The second argument `"mykernel"` is the name of the kernel function in the source code from above.
          - The source code can contain multiple kernel functions, so we need to specify the name of the kernel function we wan to use.
        - The final argument `err` is our variable for storing the error code produced by this function.
      - Finally, we call our helper function `checkOpenCL` passing in the returned error code `err`.
      
      ```c
      // 6. Kernel
      cl_kernel kernel = clCreateKernel(program, "mykernel", &err);
      checkOpenCL(err, "clCreateKernel");
      ```
    - To execute a kernel on the device, we use the OpenCL function `clEnqueueNDRangeKernel`.
      - We don't communicate directly with an OpenCL device, but instead enqueue commands in our command queue from above.
      - First we create two launch configuration parameters `global_size` and `local_size`, both of type `size_t`.
        - `global_size` is the total number of work items (threads) we want to launch when calling the kernel.
        - `local_size` is number of work items (threads) in a work group.
        - Both can be one-dimensional (1D), two-dimensional (2D), or three-dimensional (3D), depending on the problem we want to solve.
          - E.g. for working with 1D arrays, 2D arrays, or 3D arrays.
          - Each work item (thread) usually only processes 1 element in an array, but they all do so in parallel (at the same time).
          - Each work item (thread) executes a copy of the same kernel function (with the same argument values to the kernel function).
      - The call to the OpenCL function `clEnqueueNDRangeKernel` takes multiple arguments, but we'll focus on the most important ones in this notebook.
        - The first argument `queue` is our queue from above.
        - The second argument `kernel` is our kernel from above.
        - The third argument `1` is the number of dimensions in our launch configuration.
        - The fifth argument `global_size` is our variable with the total number of work items (threads) from above.
        - The sixth argument `local_size` is our variable with the number of work items (threads) in a work group from above.
        - The final argument can contain an OpenCL event (we will see how to use an event later).
      - Finally, we call our helper function `checkOpenCL` passing in the returned error code `err`.
      
      ```c
      // 7. Launch
      size_t global_size = 1;
      size_t local_size = 1;
      err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);
      checkOpenCL(err, "clEnqueueNDRangeKernel");
      ```
    - Since the call to the OpenCL function `clEnqueueNDRangeKernel` is an asynchronous call, we need to wait until all enqueued items have been processed.
      - We do this with the OpenCL function `clFinish`, passing in the `queue` we want to wait for as the single argument.
      
      ```c
      // 8. Wait for the command queue to finish
      clFinish(queue);
      ```
    - Finally, we release (deallocate) all resources we have created with a handful of OpenCL functions:
      - `clReleaseKernel` takes our `kernel` as an argument and releases it.
      - `clReleaseProgram` takes our `program` as an argument and releases it.
      - `clReleaseCommandQueue` takes our `queue` as an argument and releases it.
      - `clReleaseContext` takes our `context` as an argument and releases it.
      
      ```c
      // 9. Cleanup
      clReleaseKernel(kernel);
      clReleaseProgram(program);
      clReleaseCommandQueue(queue);
      clReleaseContext(context);
      ```
- Wow! That was a lot of boiler plate code just to launch a single kernel on a device.
  - This is bacause OpenCL is more general then e.g. CUDA, and can work with multiple types of hardware, GPU, CPU, FPGA, DSP, etc.
  - Later, we will create helper functions to setup and teardown OpenCL, and use them in this notebook to make the examples clearer.
- Additional steps, not shown above, are the steps for allocating memory on the device, copying memory between the host (CPU) and the device (GPU), etc.
  - This is done before (copying memory from the host to the device) and after (copying memory from the device to the host) a kernel launch.
- Run the cells below to see the output (the first cell writes the code to file, the second cell compiles and executes the C program with the OpenCL code).

In [ ]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>
#define CL_TARGET_OPENCL_VERSION 300
#include <CL/cl.h>

const char *source =
"__kernel void mykernel()\n"
"{\n"
"    printf(\"Hello World!\\n\");\n"
"}\n";

void checkOpenCL(cl_int err, const char *msg)
{
    if (err != CL_SUCCESS)
    {
        fprintf(stderr, "%s failed: %d\n", msg, err);
        exit(EXIT_FAILURE);
    }
}

void checkOpenCLBuildLog(cl_int err, cl_program program, cl_device_id device)
{
    if (err != CL_SUCCESS)
    {
        char log[2048];
        clGetProgramBuildInfo(program, device, CL_PROGRAM_BUILD_LOG, sizeof(log), log, NULL);
        printf("Build log:\n%s\n", log);
        exit(EXIT_FAILURE);
    }
}

int main()
{
    cl_int err;

    // 1. Platform
    cl_platform_id platform;
    cl_uint num_platforms;
    err = clGetPlatformIDs(1, &platform, &num_platforms);
    checkOpenCL(err, "clGetPlatformIDs");

    // 2. Device
    cl_device_id device;
    cl_uint num_devices;
    err = clGetDeviceIDs(platform, CL_DEVICE_TYPE_DEFAULT, 1, &device, &num_devices);
    checkOpenCL(err, "clGetDeviceIDs");

    // 3. Context
    cl_context context = clCreateContext(NULL, 1, &device, NULL, NULL, &err);
    checkOpenCL(err, "clCreateContext");

    // 4. Command queue
    cl_command_queue queue = clCreateCommandQueueWithProperties(context, device, 0, &err);
    checkOpenCL(err, "lCreateCommandQueueWithProperties");

    // 5. Program
    cl_program program = clCreateProgramWithSource(context, 1, &source, NULL, &err);
    checkOpenCL(err, "clCreateProgramWithSource");
    err = clBuildProgram(program, 1, &device, NULL, NULL, NULL);
    checkOpenCLBuildLog(err, program, device);

    // 6. Kernel
    cl_kernel kernel = clCreateKernel(program, "mykernel", &err);
    checkOpenCL(err, "clCreateKernel");

    // 7. Launch
    size_t global_size = 1;
    size_t local_size = 1;
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);
    checkOpenCL(err, "clEnqueueNDRangeKernel");

    // 8. Wait for the command queue to finish
    clFinish(queue);

    // 9. Cleanup
    clReleaseKernel(kernel);
    clReleaseProgram(program);
    clReleaseCommandQueue(queue);
    clReleaseContext(context);

    return 0;
}

Overwriting main.c


In [29]:
!{build_single_file_command}
!{execute_command}

Hello World!


### Creating a Utility Module
- Let's create a C module `utils` containing helper functions we can use in this notebook, so the examples don't get bogged down in boiler plate code.
  - `utils.c` (placed in folder `src`) contains helper function implementations.
  - `utils.h` (placed in folder `include`) contains function prototypes for the helper functions.
- Let's breifly look at what the helper functions do:
  - First we use a couple of `#define`s for the path to a file containing the source code for our OpenCL kernel, and the name of the kernel function.
    - `#define KERNEL_FILE "src/kernel.cl"` means the path to the file containing our kernel's source code will always be "src/kernel.cl"` in this notebook.
    - `#define KERNEL_NAME "mykernel"` means the name of the kernel function within that source code will always be "mykernel"` in this notebook.
    - This will make the examples clearer, so we don't have to specify this in every example with boiler plate code.
  - `read_kernel_source` takes a filename (or path) to a file containing the source code for our kernel, and loads it into a variable.
    - Note that this function allocates space on the heap, so the calling function must free the memory when done with it.
  - `openclGetErrorString` takes an OpenCL error code, and returns a string representation of it.
    - This will make it easier to interpret any errors produced in our code.
  - `checkOpenCL` is our error checking function from before, but now prints out a string represenetation of an error instead of the error code.
    - It calls the function `openclGetErrorString` above.
  - `checkOpenCLBuildLog` basically does the same as `checkOpenCL`, but also takes a `program` and `device` as arguments, and prints out compile errors.
    - This will enable us to see messages produced by the OpenCL compiler when it compiles our OpenCL kernels.
  - `setupOpenCL` takes pointers `cl_context *context`, `cl_command_queue *queue`, `cl_program *program,` `cl_kernel *kernel` as arguments and performs steps 1-6 we sw before.
    - 1. Platform - queries the system for an OpenCL platform and selects the default platform (i.e. Nvidia, AMD, or Intel).
    - 2. Device - queries the platform for OpenCL-enabled devices and selects the default device (usually a GPU if it exists, otherwise a CPU).
    - 3. Context - creates a `context`object for the device.
    - 4. Command Queue - creates a command `queue` object for the device within the `context`.
    - 5. Program - creates a `program` object and builds our OpenCL source code for the kernel (which will always be `src/kernel.cl` in this notebook).
    - 6. Kernel - create a `kernel` object from the `program` given a kernel function name (which will always be `"mykernel"` in this notebook).
    - We just need to remember to call this function at the top of our `main()` function to set things up using this boiler plate code.
      - We also need to declare the variables `cl_context context`, `cl_command_queue queue`, `cl_program program,` `cl_kernel kernel` and pass them in as arguments.
  - `teardownOpenCL` takes pointers `cl_context *context`, `cl_command_queue *queue`, `cl_program *program,` `cl_kernel *kernel` as arguments and performs the cleanup step.
    - It releases (frees) the objects `kernel`, `program`, `queue`, and `context` created by the function `setupOpenCL` above.
- Run the two cells below to create this utility module.

In [30]:
%%writefile src/utils.c
#include <stdio.h>
#include <stdlib.h>
#include "utils.h"

#define KERNEL_FILE "src/kernel.cl"
#define KERNEL_NAME "mykernel"

char* read_kernel_source(const char* filename)
{
    FILE* fp = fopen(filename, "r");
    if (!fp)
    {
        fprintf(stderr, "Failed to open kernel file: %s\n", filename);
        exit(1);
    }
    fseek(fp, 0, SEEK_END);
    long size = ftell(fp);
    rewind(fp);
    char* src = (char*)malloc(size + 1);
    fread(src, 1, size, fp);
    src[size] = '\0';
    fclose(fp);
    return src;
}

const char* openclGetErrorString(cl_int err)
{
    switch (err)
    {
        case CL_SUCCESS:                                  return "CL_SUCCESS";
        case CL_DEVICE_NOT_FOUND:                         return "CL_DEVICE_NOT_FOUND";
        case CL_DEVICE_NOT_AVAILABLE:                     return "CL_DEVICE_NOT_AVAILABLE";
        case CL_COMPILER_NOT_AVAILABLE:                   return "CL_COMPILER_NOT_AVAILABLE";
        case CL_MEM_OBJECT_ALLOCATION_FAILURE:            return "CL_MEM_OBJECT_ALLOCATION_FAILURE";
        case CL_OUT_OF_RESOURCES:                         return "CL_OUT_OF_RESOURCES";
        case CL_OUT_OF_HOST_MEMORY:                       return "CL_OUT_OF_HOST_MEMORY";
        case CL_PROFILING_INFO_NOT_AVAILABLE:             return "CL_PROFILING_INFO_NOT_AVAILABLE";
        case CL_MEM_COPY_OVERLAP:                         return "CL_MEM_COPY_OVERLAP";
        case CL_IMAGE_FORMAT_MISMATCH:                    return "CL_IMAGE_FORMAT_MISMATCH";
        case CL_IMAGE_FORMAT_NOT_SUPPORTED:               return "CL_IMAGE_FORMAT_NOT_SUPPORTED";
        case CL_BUILD_PROGRAM_FAILURE:                    return "CL_BUILD_PROGRAM_FAILURE";
        case CL_MAP_FAILURE:                              return "CL_MAP_FAILURE";
        case CL_MISALIGNED_SUB_BUFFER_OFFSET:             return "CL_MISALIGNED_SUB_BUFFER_OFFSET";
        case CL_EXEC_STATUS_ERROR_FOR_EVENTS_IN_WAIT_LIST:return "CL_EXEC_STATUS_ERROR_FOR_EVENTS_IN_WAIT_LIST";
        case CL_COMPILE_PROGRAM_FAILURE:                  return "CL_COMPILE_PROGRAM_FAILURE";
        case CL_LINKER_NOT_AVAILABLE:                     return "CL_LINKER_NOT_AVAILABLE";
        case CL_LINK_PROGRAM_FAILURE:                     return "CL_LINK_PROGRAM_FAILURE";
        case CL_DEVICE_PARTITION_FAILED:                  return "CL_DEVICE_PARTITION_FAILED";
        case CL_KERNEL_ARG_INFO_NOT_AVAILABLE:            return "CL_KERNEL_ARG_INFO_NOT_AVAILABLE";

        // clCreateProgramWithSource, clBuildProgram, etc.
        case CL_INVALID_VALUE:                            return "CL_INVALID_VALUE";
        case CL_INVALID_DEVICE_TYPE:                      return "CL_INVALID_DEVICE_TYPE";
        case CL_INVALID_PLATFORM:                         return "CL_INVALID_PLATFORM";
        case CL_INVALID_DEVICE:                           return "CL_INVALID_DEVICE";
        case CL_INVALID_CONTEXT:                          return "CL_INVALID_CONTEXT";
        case CL_INVALID_QUEUE_PROPERTIES:                 return "CL_INVALID_QUEUE_PROPERTIES";
        case CL_INVALID_COMMAND_QUEUE:                    return "CL_INVALID_COMMAND_QUEUE";
        case CL_INVALID_HOST_PTR:                         return "CL_INVALID_HOST_PTR";
        case CL_INVALID_MEM_OBJECT:                       return "CL_INVALID_MEM_OBJECT";
        case CL_INVALID_IMAGE_FORMAT_DESCRIPTOR:          return "CL_INVALID_IMAGE_FORMAT_DESCRIPTOR";
        case CL_INVALID_IMAGE_SIZE:                       return "CL_INVALID_IMAGE_SIZE";
        case CL_INVALID_SAMPLER:                          return "CL_INVALID_SAMPLER";
        case CL_INVALID_BINARY:                           return "CL_INVALID_BINARY";
        case CL_INVALID_BUILD_OPTIONS:                    return "CL_INVALID_BUILD_OPTIONS";
        case CL_INVALID_PROGRAM:                          return "CL_INVALID_PROGRAM";
        case CL_INVALID_PROGRAM_EXECUTABLE:               return "CL_INVALID_PROGRAM_EXECUTABLE";
        case CL_INVALID_KERNEL_NAME:                      return "CL_INVALID_KERNEL_NAME";
        case CL_INVALID_KERNEL_DEFINITION:                return "CL_INVALID_KERNEL_DEFINITION";
        case CL_INVALID_KERNEL:                           return "CL_INVALID_KERNEL";
        case CL_INVALID_ARG_INDEX:                        return "CL_INVALID_ARG_INDEX";
        case CL_INVALID_ARG_VALUE:                        return "CL_INVALID_ARG_VALUE";
        case CL_INVALID_ARG_SIZE:                         return "CL_INVALID_ARG_SIZE";
        case CL_INVALID_WORK_DIMENSION:                   return "CL_INVALID_WORK_DIMENSION";
        case CL_INVALID_WORK_GROUP_SIZE:                  return "CL_INVALID_WORK_GROUP_SIZE";
        case CL_INVALID_WORK_ITEM_SIZE:                   return "CL_INVALID_WORK_ITEM_SIZE";
        case CL_INVALID_GLOBAL_OFFSET:                    return "CL_INVALID_GLOBAL_OFFSET";
        case CL_INVALID_EVENT_WAIT_LIST:                  return "CL_INVALID_EVENT_WAIT_LIST";
        case CL_INVALID_EVENT:                            return "CL_INVALID_EVENT";
        case CL_INVALID_OPERATION:                        return "CL_INVALID_OPERATION";
        case CL_INVALID_GL_OBJECT:                        return "CL_INVALID_GL_OBJECT";
        case CL_INVALID_BUFFER_SIZE:                      return "CL_INVALID_BUFFER_SIZE";
        case CL_INVALID_MIP_LEVEL:                        return "CL_INVALID_MIP_LEVEL";
        case CL_INVALID_GLOBAL_WORK_SIZE:                 return "CL_INVALID_GLOBAL_WORK_SIZE";        
        default:
        {
            static char unknown[64];
            snprintf(unknown, sizeof(unknown), "Unknown OpenCL error code %d", err);
            return unknown;
        }
    }
}

void checkOpenCL(cl_int err, const char *msg)
{
    if (err != CL_SUCCESS)
    {
        printf("Error: %s (%s)\n", msg, openclGetErrorString(err));
        exit(EXIT_FAILURE);
    }
}

void checkOpenCLBuildLog(cl_int err, cl_program program, cl_device_id device)
{
    if (err != CL_SUCCESS)
    {
        char log[2048];
        clGetProgramBuildInfo(program, device, CL_PROGRAM_BUILD_LOG, sizeof(log), log, NULL);
        printf("Build log:\n%s\n", log);
        exit(EXIT_FAILURE);
    }
}

void setupOpenCL(cl_context *context, cl_command_queue *queue, cl_program *program, cl_kernel *kernel)
{
    cl_int err;
    
    // 1. Platform
    cl_platform_id platform;
    cl_uint num_platforms;
    err = clGetPlatformIDs(1, &platform, &num_platforms);
    checkOpenCL(err, "clGetPlatformIDs");

    // 2. Device
    cl_device_id device;
    cl_uint num_devices;
    err = clGetDeviceIDs(platform, CL_DEVICE_TYPE_DEFAULT, 1, &device, &num_devices);
    checkOpenCL(err, "clGetDeviceIDs");

    // 3. Context
    /*cl_context*/ *context = clCreateContext(NULL, 1, &device, NULL, NULL, &err);
    checkOpenCL(err, "clCreateContext");

    // 4. Command queue
    const cl_queue_properties props[] = { CL_QUEUE_PROPERTIES, CL_QUEUE_PROFILING_ENABLE, 0 };
    /*cl_command_queue*/ *queue = clCreateCommandQueueWithProperties(*context, device, props, &err);
    checkOpenCL(err, "lCreateCommandQueueWithProperties");

    // 5. Program
    char* source = read_kernel_source(KERNEL_FILE); // Read kernel source
    /*cl_program*/ *program = clCreateProgramWithSource(*context, 1, (const char**)&source, NULL, &err);
    checkOpenCL(err, "clCreateProgramWithSource");
    err = clBuildProgram(*program, 1, &device, NULL, NULL, NULL);
    checkOpenCLBuildLog(err, *program, device);

    // 6. Kernel
    /*cl_kernel*/ *kernel = clCreateKernel(*program, KERNEL_NAME, &err);
    checkOpenCL(err, "clCreateKernel");
}

void teardownOpenCL(cl_context *context, cl_command_queue *queue, cl_program *program, cl_kernel *kernel)
{
    clReleaseKernel(*kernel);
    clReleaseProgram(*program);
    clReleaseCommandQueue(*queue);
    clReleaseContext(*context);
}

Writing src/utils.c


In [31]:
%%writefile include/utils.h
#pragma once
#define CL_TARGET_OPENCL_VERSION 300
#include <CL/cl.h>

char* read_kernel_source(const char* filename);
const char* openclGetErrorString(cl_int err);
void checkOpenCL(cl_int err, const char *msg);
void checkOpenCLBuildLog(cl_int err, cl_program program, cl_device_id device);
void setupOpenCL(cl_context *context, cl_command_queue *queue, cl_program *program, cl_kernel *kernel);
void teardownOpenCL(cl_context *context, cl_command_queue *queue, cl_program *program, cl_kernel *kernel);

Writing include/utils.h


### Using the Utility Module
- Now we can create:
  - The source code for our OpenCL kernel in a file `src/kernel.cl` instead of embedding it as a C string inside our code.
  - Our `main()` function without all the boiler plate code, and place it in a file `src/main.c`.
  - All examples in this notebook will use the following skeleton code.
    - We include any header files we need, including the utility module we just created `#include "utils.h"`.
    - Then we create a variable `err` of type `cl_int` to store any error codes.
    - Next, we create variables `cl_context context`, `cl_command_queue queue`, `cl_program program`, and `cl_kernel kernel`.
     - Then, we pass the variables' addresses to our helper function `setupOpenCL`, which performs steps 1-6 in the boiler plate code.
    - Then we have the code to launch our kernel (it has already been loaded and compiled by the boiler plate code).
      - It will always be the kernel function called `mykernel` in the file `src/kernel.cl`.
      - In the notebook examples, before launching a kernel, we will:
        - Allocate and initialize memory on the host (CPU).
        - Allocated memory on the device (GPU) and copy memory between the host (CPU) and device (GPU).
        - Setup launch configuration parameters and the arguments we want to pass to the kernel function.
      - We then wait for the kernel function to compelte (we can do that with the function `clFinish` as below, or use other relevant functions).
        - This will block the `main()` function's main thread since this function call is synchronous.
      - In the notebook examples, after waiting for the kernel to finish, we will:
        - Copy memory between the device (GPU) and the host (CPU).
        - Inspect the results from our kernel computation.
      - Finally, we call our helper function `teardownOpenCL`, which releases the `context`, `queue`, `program`, and `kernel`. in the boiler plate code.
        - When we create other OpenCL objects in our example code, we have to remeber to release these before calling this helper function.

    ```c
    #include <stdio.h>
    #include <stdlib.h>
    #include "utils.h"

    int main()
    {
        cl_int err;

        // 1. Platform, 2. Device, 3. Context, 4. Command queue, 5. Program, 6. Kernel
        cl_context context;
        cl_command_queue queue;
        cl_program program;
        cl_kernel kernel;
        setupOpenCL(&context, &queue, &program, &kernel);

        // 7. Launch
        size_t global_size = 1;
        size_t local_size = 1;
        err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);
        checkOpenCL(err, "clEnqueueNDRangeKernel");

        // 8. Wait and finish
        clFinish(queue);

        // 9. Cleanup
        teardownOpenCL(&context, &queue, &program, &kernel);

        return 0;
    }
    ```
- Run the three cells below to create the kernel `src/kernel.cl`, create and build the main program `src/main.c`, and run the resulting executable `bin/main.exe`.
  - The example code is a lot clearer now that it isn't cluttered with all the boiler plate code.

In [32]:
%%writefile src/kernel.cl
__kernel void mykernel()
{
    printf("Hello World!\n");
}

Overwriting src/kernel.cl


In [33]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include "utils.h"

int main()
{
    cl_int err;

    // 1. Platform, 2. Device, 3. Context, 4. Command queue, 5. Program, 6. Kernel
    cl_context context;
    cl_command_queue queue;
    cl_program program;
    cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    // 7. Launch
    size_t global_size = 1;
    size_t local_size = 1;
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);
    checkOpenCL(err, "clEnqueueNDRangeKernel");

    // 8. Wait and finish
    clFinish(queue);

    // 9. Cleanup
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [34]:
!{build_multi_file_command}
!{execute_command}

Hello World!


---
## 2.4 NDRange (Global Size), Work Groups, Work Items, Devices, CUs, and PEs

- In the previous OpenCL code, we saw:
  - An OpenCL **kernel**, where:
    - `__kernel` is an OpenCL-specific qualifier that makes a function a **kernel function**.
      - A **kernel function** is executed on the device (GPU), and called from the host (CPU).
    - `mykernel` is the name of the **kernel function**.
    - `parameterlist` is the kernel function's comma-separated list of parameters.

      ```c
      __kernel kernel(parameterlist)
      {

      }
      ```
  - An OpenCL **kernel launch** (or really an enqueuement on the command queue), where:
    - `queue` is the command queue object.
    - `kernel` is the kernel object.
    - `1` is the number of dimensions in our launch configuration (`1` for a 1D problem, `2` for a 2D problem, `3` for a 3D problem).
    - `global_size` is a **launch parameter** with the **total number of `work items`** (threads).
      - E.g. for a 2D problem we would set the parameter above to `2`, and define our variable as `size_t global_size[] = {<dim0>, <dim1>};`.
    - `local_size` is a **launch parameter** with the **number of `work items` in a `work group`**.
      - E.g. for a 2D problem we would set the parameter above to `2`, and define our variable as `size_t local_size[] = {<dim0>, <dim1>};`.
    - What's missing is the `argumentlist`, i.e. the arguments to pass to the kernel function.
      - These are set separately, before the kernel launch, with a series of calls to the OpenCL function `clSetKernelArg(kernel, index, size, value)` where:
        - `kernel` is the kernel object.
        - `index` is the ordinal position of the parameter in the kernel function, e.g. `0` for the first parameter, `1` for the second parameter, etc.
        - `size` is the size of the argument in bytes, e.g. `sizeof(int)` for a variable `n` containing the number of elements in an `int` array, 
        - `value` is the address to the variable, e.g. `&n` for a variable containing the number of elements in an array. 
    
      ```bash
      size_t global_size = 1;
      size_t local_size = 1;
      err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &global_size, &local_size, 0, NULL, NULL);
      checkOpenCL(err, "clEnqueueNDRangeKernel");
      ```

<img src="images/opencl.png" width="500" style="float: right; margin-right: 50px;" />

- An OpenCL-enabled device (e.g. GPU):
  - Is called a `Device`.
  - Has a number of `Compute Units (CUs)`.
  - Each `CU` has a number of `Processing Elements (PEs)`.
- During a **kernel launch**:
  - OpenCL lauches an `NDRange` (basically a grid). 
  - An `NDRange` contains a total number of `work items` (threads), i.e. the `Global Size`, specified as `global_size` in the call to `clEnqueueNDRangeKernel`.
  - The total number of `work items` (threads) are grouped into a number of `work groups`.
  - Each `work group` contains a number of `work items` (threads), specified as `local_size` in the call to `clEnqueueNDRangeKernel`.
- OpenCL maps:
  - A `work group` to run on a `CU` (multiple `work groups` can be assigned to the same `CU`).
  - Each `work item` (thread) within a `work group` to a `PE`.
- So, a `work group` runs on a `CU`, and each `work item` (thread) within that `work group` runs on a `PE` within that `CU`.


int num_work_groups = get_num_groups(0); // equivalent to CUDA's gridDim.x
int work_group_size = get_local_size(0); // equivalent to CUDA's blockDim.x
int work_group_id = get_group_id(0);     // equivalent to CUDA's blockIdx.x
int work_item_id = get_local_id(0);      // equivalent to CUDA's threadIdx.x
int idx = get_global_id(0);              // equivalent to CUDA's threadIdx.x + blockIdx.x * blockDim.x

- For each **kernel launch**:
  - OpenCL creates an `NDRange` containing `work groups` containing `work items` (threads).
  - Each `work item` (thread) runs a copy of the same `kernel` function with the same `argumentlist`.
  - 4 OpenCL-specific functions are available within each copy of the `kernel` function:
    - `get_num_groups(dim)` returns the number of `work groups` in the kernel launch for the given dimension `dim`,
      - Where `dim` can be `0`, `1`, and `2`, depending on the kernel launch parameters' dimensions.
    - `get_local_size(dim)` returns the number of `work items` (threads) in a `work group` for the given dimension `dim`,
      - Where `dim` can be `0`, `1`, and `2`, depending on the kernel launch parameters' dimensions.
    - `get_group_id(dim)` returns the ID of a `work group`, unique within a kernel launch per dimension, for the given dimension `dim`,
      - Where `dim` can be `0`, `1`, and `2`, depending on the kernel launch parameters' dimensions.
    - `get_local_id(dim)` returns the ID of a `work item` (thread), unique withn a `work group` per dimension, for the given dimension `dim`,
      - Where `dim` can be `0`, `1`, and `2`, depending on the kernel launch parameters' dimensions.
    - `get_gloabl_id(dim)` returns the ID of the `work item` (thread), unique wthin a kernel launch per dimension, for the given dimension `dim`,
      - Where `dim` can be `0`, `1`, and `2`, depending on the kernel launch parameters' dimensions.
    
  - In the figure, we see that each `CU` contains:
    - A `register file` (the blue rectangle).
      - The `register file` is divided into `**chunks**, where each `work item` (thread), running in the `work group` assigned to the `CU`, is assigned one **chunk**.
      - Each `work item`'s (thread's) chunk of the `register file` is referred to as its `private memory` (only that `work item` (thread) can access that `private memory`).
      - In a `kernel` function, we can declare a local variable with the qualifier `__private` which stores that variable in a `work item`'s (thread's) `private memory`.
        - If we don't specify a qualifier, the variable is by default stored in the `work items`'s (thread's) `private memory`.
    - A `shared memory` buffer (the green rectangle).
      - The `shared memory` is shared by all `work items` (threads) running in the `work group` assigned to that `CU`.
      - To store a variable in `shared memory`, we declare the variable with the `__local` qualifier.
  - There is also `global memory`, which is declared using the qualifier `__global`.
  - Finally, there is `constant memory` stored outside a `CU` (just like `global memory`), and is `read-only` memory (it can we written to once before a kernel launch).
    - If we declare a variable with the qualifier `__constant`, it is placed in `constant memory`.
    - `constant memory` is small, but very effient (the OpenCL compiler can optimize access to it since it is `read-only`).

### Code Demonstrating NDRanges, Work Groups, and Work Items

- Let's write an OpenCL program that demonstrates running an `NDRange` of `work groups`, each with a number of `work items` (threads), on the device (GPU).
- The program copies the elements from one 1D `int` array `input` to another 1D `int` array `output`.
  - We include the necessary header files:
    - `stdio.h` for `printf`
    - `stdlib.h` for `malloc` and `free`
    - `time.h` for `srand` and `rand`
    - `utils.h` for our helper functions, which also includes `Cl/cl.h` with function prototypes for all the OpenCL functions, types, etc.
    
    ```c
    #include <stdio.h>
    #include <stdlib.h>
    #include <time.h>
    #include "utils.h"
    ```
  
  - We define symbolic constants:
    - `N` with the value `5` for the number of elements in each array
    - `WORKITEMS_PER_WORKGROUP_0` with the value `2` for the number of `work items` (threads) in each `work group`

    ```c
    #define N 5
    #define WORKITEMS_PER_WORKGROUP_0 2
    ```
  
  - We define a kernel function called `mykernel`.
    - It has the `__kernel` qualifier, making it a kernel function, returns `void`, and has three parameters:
      - A `const int *` pointer `input` which points to the `input` array in the device's `global memory`, since we use the qualifier `__gloabal`.
      - An `int *` pointer `output` which points to the `output` array in the device's `global memory`, since we use the qualifier `__gloabal`.
      - An `int` variable `n` with the number of elements in each array.
    - In the function's body, we:
      - Use the OpenCL-specific functions `get_num_groups`, `get_local_size`, `get_group_id`, `get_local_id` and `get_global_id`.
        - They are available to each `work item` (thread) within each copy of the kernel function.
        - `get_num_groups(0)` returns the number of `work groups` in the kernel launch for dimension `0`
          - In our case we have `3` since we calculate it in the `main` function as:

            ```c
            size_t localSize = WORKITEMS_PER_WORKGROUP_0; // 2
            size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0; // ((5 + 2 - 1) / 2) * 2 = 6
            size_t number_of_work_groups = globalSize / localSize // 6 / 2 = 3
            ```
        - `get_local_size(0)` returns the number of `work items` (threads) in a `work group` for dimension `0`,
          - In our case we have `2`, defined by `WORKITEMS_PER_WORKGROUP_0`.
        - `get_group_id(0)` returns the ID of a `work group`, unique within a kernel launch per dimension, for the dimension `0`,
          - Since the number of `work groups` is `globalSize / localSize = 3`, `get_group_id(0)` will range from `0` to `2` (zero-based, i.e. starting from `0`).
        - `get_local_id(0)` returns the ID of a `work item` (thread), unique withn a `work group` per dimension, for dimension `0`,
          - Since `localSize = WORKITEMS_PER_WORKGROUP_0 = 2`, `get_local_id(0)` ranges from `0` to `1` (zero-based, i.e. starting from `0`).
        - `get_gloabl_id(0)` returns the ID of the `work item` (thread), unique wthin a kernel launch per dimension, for dimension `0`,
          - Since `gloablSize = 6`, `get_global_id(0)` ranges from `0` to `5` (zero-based, i.e. starting from `0`).
      - We print out the values returned from these functions.
        - This shows us which `work item` (thread) is running this specific copy of the kernel function, which `work group` it is in, etc.
      - We have a `boundary guard`, i.e. `if(idx >= n) return;`
        - This ensures we don't index into an array with `idx` if `idx` is out of bounds.
        - We have `6` threads in total, and `5` (defined by `N`) elements in each array, so `idx = 5` is out of bounds.
      - Finally, we copy one element from the `input` array into the `output` using `idx` as the index.
        - Remember, each thread runs its own `copy` of the kernel function (in parallel), each with the same set of kernel function `arguments`.

    ```c
    __kernel void mykernel(__global const int *input, __global int *output, int n)
    {
        int num_work_groups = get_num_groups(0); // equivalent to CUDA's gridDim.x
        int work_group_size = get_local_size(0); // equivalent to CUDA's blockDim.x
        int work_group_id = get_group_id(0);     // equivalent to CUDA's blockIdx.x
        int work_item_id = get_local_id(0);      // equivalent to CUDA's threadIdx.x
        int idx = get_global_id(0);              // equivalent to CUDA's threadIdx.x + blockIdx.x * blockDim.x

        printf("num_work_groups = %d, work_group_size = %d, work_group_id = %d, work_item_id = %d, idx = %d\n", num_work_groups, work_group_size, work_group_id, work_item_id, idx);

        if (idx >= n)
        {
            printf("Boundary checking avoided indexing outside of the arrays [idx = %d]\n", idx);
            return;
        }

        output[idx] = input[idx];
    }
    ```
- In the `main()` function:
  - We declare our OpenCL variables and call our help function `setupOpenCL` as previously described.

    ```c
    // Setup OpenCL
        cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
        setupOpenCL(&context, &queue, &program, &kernel);
    ```
  - We seed the pseudorandom number generator with the value `0` so the random numers we create will be the same every time we run the program.

    ```c
    srand(0);
    ```

  - We declare:
    - `int` pointer variables `h_input` and `h_output` for the two arrays, which will point to heap memory (RAM) on the host (CPU).
    - `int` variable `data_size` and initialize it to `N * sizeof(int)`, i.e. the total size of each array in bytes (with `N` elements of type `int` in each).
    
    ```c
    int *h_input, *h_output;
    int data_size = N * sizeof(int);
    ```
  - We allocate memory on the host (CPU) with `malloc`, storing the pointers to the memory in variables `h_input` and `h_output`.

    ```c
    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);
    ```
  - We initialize the `h_input` array on the host (CPU) with random values using the `rand()` function.

    ```c
    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100; // random integers between 0 and 99
    }
    ```
  - We allocate memory on the device (GPU) with `clCreateBuffer` storing the pointers to the memory in variables `d_input` and `d_output`, both of type `cl_mem`.
    - `clCreateBuffer` also copies values from host (CPU) mmeory to device (GPU) memory if we specify the `CL_MEM_COPY_HOST_PTR` as a flag.
      - This is done for the `input` vector.
    - We can also specify is the memory is read-only with flag `CL_MEM_READ_ONLY`, write-only with flag `CL_MEM_WRITE_ONLY` (or both with `CL_MEM_READ_WRITE`).
      - The `input` vector is read-only and the `output` vector is write-only on the device (GPU).
    - The flags are bitwise or:ed together and passed as the second argument to the function.
    - We also recognize the first argument `context` and the last argument `err`.
    - The third argument is the size of the memory in bytes for the `input` vector.
    - The fourth argument is the `input` vector on the host (CPU).
    - The return is a pointer to the `input` vector on the device (GPU).
    
    ```c
    // Device buffers
    cl_mem d_input, d_output;
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, N * sizeof(int), h_input, &err); // CL_MEM_COPY_HOST_PTR copies h_input values to device buffer
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, N * sizeof(int), NULL, &err);
    ```
  - Next, we set the arguments passed to the kernel function, which we do using the `clSetKernelArg` function.
    - The signature of our kernel function is `__kernel void mykernel(__global const int *input, __global int *output, int n)`, so we need to set three arguments.
      - The first parameter to `clSetKernelArg` is the `kernel` object.
      - The second parameter to `clSetKernelArg` is the argument index, e.g. `0` for the first argument, `1` for the second argument, etc.
      - The third parameter to `clSetKernelArg` is the size in bytes of what we are passing to the kernel's parameter.
      - The fourth parameter to `clSetKernelArg` is the address to the variable containg the value we want to pass the the kernel function's parameter.
    - Here we are passing the pointers to the `input` and `output` vectors on the device (GPU) to the kernel function, together with the number of elements.
    
    ```c
    // Set kernel arguments
    cl_int n = N;
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    clSetKernelArg(kernel, 2, sizeof(int), &n);
    ```
  - Next, we create and initialize variables `localSize` and `globalSize` for the launch configuration, both of type `size_t`.
    - Here, we are assigning integers to them since we are only using 1 dimension, but we can also declare them as arrays, e.g. `localSize[2]` or `localSize[3]`.
    - The launch configuration supports launching `1D`, `2D`, and `3D` work groups and work items (threads), depending on the problem we want to solve, e.g:
      - For a `1D` problem such as copying elements between two 1D arrays, we only need to use 1 dimension, which is dimension `0` in the `get_` prefixed functions above.
      - For a `2D` problem such as filtering a 2D image, we might need to use 2 dimensions, which are dimensions `0` and `1` in the `get_` prefixed functions above.
      - For a `3D` problem such as filtering a 3D MRI-scan volume, we might need to use 3 dimensions, which are dimensions `0`, `1` and `z` in the `get_` prefixed functions above.
      - For our 1D problem, we are only using one dimension `0`, which means the other dimensions `1` and `2` aren't available in the `get_` prefixed functions above.
    - We create a `dim3` variable `blockDim` and initialize its `x` member variable to `THREADS_PER_BLOCK_X` (member variables `y` and `z` will be set to `1`).
      - This means we have `2` threads per block since `THREADS_PER_BLOCK_X` is defined with the value `2`.
    - We create a `size_t` variable `localSize` and initialize it to `WORKITEMS_PER_WORKGROUP_0` which is `2` in our example.
    - We create a `size_t` variable `globalSize` and initialize it to:
      - `((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0` which is `((5 + 2 -1) / 2) * 2 = 6` in our example.
      - So, the total number of `work items` (threads) is `globalSize = 6`, and the number of `work items` (threads) per `work group` is `localSize = 2`.
      - This means the number of `work groups` is `globalSize / localSize = 6 / 2 = 3`.
      - This (standard) construct is commonly used to ensure enough `work items` (threads) and `work groups` are launched to solve a problem.
        - But it can launch more `work items` (threads) than there are data elements (`N = 5` in our example).
        - So we need a boundary guard in the kernel to prevent out-of-bounds indexing outside of the arrays.

    ```c
    // Kernel launch configuration
    size_t localSize = WORKITEMS_PER_WORKGROUP_0;
    size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0;
    ```
  - Next, we launch the kernel (or really enqueue it on the command queue), be calling the `clEnqueueNDRangeKernel` function, passing in the necessary arguments:
    - The first argument is our `queue` object.
    - The second argument is our `kernel` object.
    - The third argument is the number of dimensions we are using in our launuch configuration, which is `1` in our case.
    - The fifth argument is our `globalSize` variable, i.e. the total number of `work items` (threads) which is `6` in our case.
    - The sixth argument is our `localSize` variable, i.e. the number of `work items` (threads) in each `work group` which is `2` in our case.
    
    ```c
    // Enqueue kernel
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, NULL);
    ```
  - We copy the elements in the `d_output` array on the device (GPU) back to the array `h_output` on the host (CPU) using `clEnqueueReadBuffer`.
    - The first argument is our `queue` object.
    - The second argument is our pointer `d_output` to memory on the device (GPU).
    - The third argument is the flag `CL_TRUE` which makes this function call asynchronous (it blocks the `main` function's main thread until the memory has been copied).
    - The fifth argument is the size in bytes that we want to copy, in our case `N * sizeof(int) = 5 * 32 = 160` (on most systems).
    - The sixth argument is our pointer `h_output` to memory on the host (CPU).

    ```c
    // Read result back (CL_TRUE makes this a synchronous (blocking) call)
    err = clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, N * sizeof(int), h_output, 0, NULL, NULL);
    ```
  - We also have wait for the command queue' enqueued items to finish, i.e. `clFinish` is a synchonous (blocking) function call.
    - In this case, it isn't really necessary since `clEnqueueReadBuffer` is a synchronous (blocking) function call due to the `CL_TRUE` flag.

    ```c
    // Wait for all queued operations to finish (not really needed here because of CL_TRUE in clEnqueueReadBuffer above)
    err = clFinish(queue);
    ```
  - Then we print out the elements in the two arrays `h_input` and `h_output` on the host (CPU).
   
    ```c
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    ```
  - Now we are done, so we free the memory allocated for the arrays:
    - We free the `int` pointers (`h_input` and `h_output`) pointing to memory on the host (CPU) with `free`.
    - We realse (free) the `int` pointers (`d_input` and `d_output`) pointing to memory on the device (GPU) with `clReleaseMemObject`.
    - Both functions take a pointer to the memory 
    - Notice the naming convention used in this program for pointers to memory on the host (`h_` prefix) and the device (`d_` prefix).

    ```c
    // Cleanup
    free(h_input);
    free(h_output);
    err = clReleaseMemObject(d_input);
    err = clReleaseMemObject(d_output);
    ```
  - Finally, we call our helper function `teardownOpenCL`, passing in the address to our `context`, `queue`, `program`, and `kernel` objects.
    - This frees (releases) these objects via our hidden boiler plate code.

    ```c
    // Cleanup
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);
    ```
- Run the three cells below to see the output from the program.

In [35]:
%%writefile src/kernel.cl
__kernel void mykernel(__global const int *input, __global int *output, int n)
{
    int num_work_groups = get_num_groups(0); // equivalent to CUDA's gridDim.x
    int work_group_size = get_local_size(0); // equivalent to CUDA's blockDim.x
    int work_group_id = get_group_id(0);     // equivalent to CUDA's blockIdx.x
    int work_item_id = get_local_id(0);      // equivalent to CUDA's threadIdx.x
    int idx = get_global_id(0);              // equivalent to CUDA's threadIdx.x + blockIdx.x * blockDim.x

    printf("num_work_groups = %d, work_group_size = %d, work_group_id = %d, work_item_id = %d, idx = %d\n", num_work_groups, work_group_size, work_group_id, work_item_id, idx);

    if (idx >= n)
    {
        printf("Boundary checking avoided indexing outside of the arrays [idx = %d]\n", idx);
        return;
    }

    output[idx] = input[idx];
}

Overwriting src/kernel.cl


In [36]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "utils.h"

#define N 5
#define WORKITEMS_PER_WORKGROUP_0 2

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    srand(0);
    
    int *h_input, *h_output;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    // Device buffers
    cl_mem d_input, d_output;
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, N * sizeof(int), h_input, &err); // CL_MEM_COPY_HOST_PTR copies h_input values to device buffer
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, N * sizeof(int), NULL, &err);

    // Set kernel arguments
    cl_int n = N;
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    clSetKernelArg(kernel, 2, sizeof(int), &n);

    // Kernel launch configuration
    size_t localSize = WORKITEMS_PER_WORKGROUP_0;
    size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0;

    // Enqueue kernel
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, NULL);

    // Read result back (CL_TRUE makes this a synchronous (blocking) call)
    err = clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, N * sizeof(int), h_output, 0, NULL, NULL);

    // Wait for all queued operations to finish (not really needed here because of CL_TRUE in clEnqueueReadBuffer above)
    err = clFinish(queue);

    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }

    // Cleanup
    free(h_input);
    free(h_output);
    err = clReleaseMemObject(d_input);
    err = clReleaseMemObject(d_output);
    
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [37]:
!{build_multi_file_command}
!{execute_command}

num_work_groups = 3, work_group_size = 2, work_group_id = 2, work_item_id = 0, idx = 4
num_work_groups = 3, work_group_size = 2, work_group_id = 2, work_item_id = 1, idx = 5
num_work_groups = 3, work_group_size = 2, work_group_id = 1, work_item_id = 0, idx = 2
num_work_groups = 3, work_group_size = 2, work_group_id = 1, work_item_id = 1, idx = 3
num_work_groups = 3, work_group_size = 2, work_group_id = 0, work_item_id = 0, idx = 0
num_work_groups = 3, work_group_size = 2, work_group_id = 0, work_item_id = 1, idx = 1
Boundary checking avoided indexing outside of the arrays [idx = 5]

input   output
83      83    
86      86    
77      77    
15      15    
93      93    


### Inspecting the Output

- In the output we see:
  - `get_num_groups(0)` is `3`, i.e. there are `3` `work groups` in the `NDRange` (grid).
  - `get_local_size(0)` is `2`, i.e. there are `2` `work items` (threads) in each `work group`.
  - `get_group_id(0)` varies from `0` to `2`, i.e. from `0` to `get_num_groups(0) - 1`, and is a `work group`'s unique ID (i.e. unique within a kernel launch).
  - `get_local_id(0)` varies from `0` to `1`, i.e. from `0` to `get_local_size(0) - 1`, and is a `work item`'s (thread's) unique block ID (i.e. unique within a work group).
  - `idx = get_global_id(0)` varies from `0` to `5`, i.e. from `0` to `get_num_groups(0) * get_local_size(0) - 1`, and is a `work item`'s (trheads) unique global ID (i.e. unique within a kernel launch).
  - The boundary guard was triggered for one work item (thread), i.e. the work item (thread) with `idx = 5`, because we only have `N = 5` elements in each array.
    - So, we can have more work items (threads) running than elements in our data/arrays, why **we should always make use of boundary guards in our OpenCL kernels**.
  - The `input` and `output` arrays have the same element values, so our OpenCL kernel's logic is functionally correct.

---
## 2.5 Error Checking

- OpenCL supports checking for errors in device (GPU) code and from calling any OpenCL function.
- We have already seen how this works in our helper functions and by using the error code of type `cl_int` returned by, or set within, an OpenCL function call.
- This example uses our helper function `checkOpenCL(err, "Message")` after each OpenCL function call to check for any errors.
  - It's exactly the same code before but with error-checking in place.
  - We have also reduced the code in the kernel function to just copy the output array to the input arrary, since we now know how work groups and work items work.
  - Also notice the code just before we call our helper function `teardownOpenCL`.
    - Here we are intentionally creating an error to see the output from our helper function `checkOpenCL`.
    - We try to set a kernel argument with index `99`, but our kernel function only has three parameters, which produces an error.

    ```c
    err = clSetKernelArg(kernel, 99, sizeof(cl_mem), &d_input);        // Intentionally set a kernel argument with invalid arg index
    checkOpenCL(err, "Intentional clSetKernelArg with invalid index");
    ```
- Inspect the code in the cells below, then run them to see the output.

In [38]:
%%writefile src/kernel.cl
__kernel void mykernel(__global const int *input, __global int *output, int n)
{
    int idx = get_global_id(0);
    output[idx] = input[idx];
}

Overwriting src/kernel.cl


In [39]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "utils.h"

#define N 5
#define WORKITEMS_PER_WORKGROUP_0 2

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    srand(0);
    
    int *h_input, *h_output;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    // Device buffers
    cl_mem d_input, d_output;
    
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, N * sizeof(int), h_input, &err); // CL_MEM_COPY_HOST_PTR copies h_input values to device buffer
    checkOpenCL(err, "clCreateBuffer");
    
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, N * sizeof(int), NULL, &err);
    checkOpenCL(err, "clCreateBuffer");

    // Set kernel arguments
    cl_int n = N;
    err = clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    checkOpenCL(err, "clSetKernelArg");
    
    err = clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    checkOpenCL(err, "clSetKernelArg");
    
    err = clSetKernelArg(kernel, 2, sizeof(int), &n);
    checkOpenCL(err, "clSetKernelArg");

    // Kernel launch configuration
    size_t localSize = WORKITEMS_PER_WORKGROUP_0;
    size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0;

    // Enqueue kernel
    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, NULL);
    checkOpenCL(err, "clEnqueueNDRangeKernel");

    // Read result back (CL_TRUE makes this a synchronous (blocking) call)
    err = clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, N * sizeof(int), h_output, 0, NULL, NULL);
    checkOpenCL(err, "clEnqueueReadBuffer");

    // Wait for all queued operations to finish (not really needed here because of CL_TRUE in clEnqueueReadBuffer above)
    err = clFinish(queue);
    checkOpenCL(err, "clFinish");

    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    printf("\n");

    // Cleanup
    free(h_input);
    free(h_output);

    err = clReleaseMemObject(d_input);
    checkOpenCL(err, "clReleaseMemObject");
    
    err = clReleaseMemObject(d_output);
    checkOpenCL(err, "clReleaseMemObject");

    err = clSetKernelArg(kernel, 99, sizeof(cl_mem), &d_input);        // Intentionally set a kernel argument with invalid arg index
    checkOpenCL(err, "Intentional clSetKernelArg with invalid index");
    
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [40]:
!{build_multi_file_command}
!{execute_command}


input   output
83      83    
86      86    
77      77    
15      15    
93      93    

Error: Intentional clSetKernelArg with invalid index (CL_INVALID_ARG_INDEX)


### Inspecting the Output

- In the output we see that the result is the same as before (the only difference is that we are NOT printing out `work groups`, `work items`, etc.).
- We also see the error `CL_INVALID_ARG_INDEX` returned from `clSetKernelArg()` when we try to set an argument with an index that doesn't exist.

---
## 2.6 Measuring Execution Time on the Host (CPU) and on the Device (GPU)

- A common workflow is to first implement an algorithm in a function on the host (CPU), and then in a kernel on the device (GPU).
  - The CPU version can act as a baseline benchmark for GPU kernel performance.
  - The CPU version can be used to verify the results of a GPU kernel.
  - For inexperienced manycore programmers, it's often easier to start with a CPU version, and then convert it into a GPU version.
- Let's use the same code as before, but instrument it with timing code, wrapped around the CPU function call and around the GPU kernel launch.
- The imported header files are the same as before:
  - `stdlib.h` also contains the function prototype for `clock`, the `clock_t` typedef, and the symbolic constant `CLOCKS_PER_SEC`.
    - `clock()` is a parameterless function returning a value of type `clock_t`.
    - `clock_t` contains the number of `ticks` elapsed since the program started.
    - `CLOCKS_PER_SEC` is defined as the number of `ticks` in a second (`ticks / CLOCKS_PER_SEC * 1000.0` converts `ticks` to milliseconds).
  - `Cl/cl.h` contains a typedef `cl_event` and prototypes `clWaitForEvents` and `clGetEventProfilingInfo`.
    - `cl_event` represent an OpenCL event, e.g. `cl_event kernel_event` (we won't explore OpenCL events in detail in this notebook).
    - `clWaitForEvents` is used wait for an event to finish (it blocks until it does), e.g. `clWaitForEvents(1, &kernel_event)`.
    - `clGetEventProfilingInfo` is get profiling information, which we use in the example to record a start and stop time (when the kernel started and stopped).
    
- We define a host (CPU) function `copy()`, equivalent to the device (GPU) kernel function `mykernel()`
  - The GPU kernel function is the same as before.
    
    ```c
    void copy(int *input, int *output, int n)
    {
        for(int idx = 0; idx < n; idx++)
        {
            output[idx] = input[idx];
        }
    }
    ```
- In the `main()` function:
  - We wrap the code below around the device (GPU) kernel launch `clEnqueueNDRangeKernel()`.
    - First we declare an OpenCL event variable `kernel_event` of type `cl_event`.
    - Then we launch (enqueue) the kernel with the kernel event as the final argument.
    - Next, we wait for the event to finish (it finished when the kernel finishes), with `clWaitForEvents(1, &kernel_event)`.
    - Then we declare two variables `time_start` and `time_end` of type `cl_ulong`.
    - We call the function `clGetEventProfilingInfo` passing in the first variable `time_start`, together with the variable `kernel_event`.
    - We call the function `clGetEventProfilingInfo` passing in the second variable `time_stop`, together with the variable `kernel_event`.
    - The flags `CL_PROFILING_COMMAND_START` and `CL_PROFILING_COMMAND_END` record when the event started and stopped, respectively.
    - Finally, we calculate the kernel's execution time, in milliseconds, using the variables time_start` and time_stop`.

    ```c
    // --------------------------------------------------------------
    // Timing the device (GPU) kernel execution time
    // --------------------------------------------------------------
    // Enqueue kernel with event
    cl_event kernel_event;
    clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, &kernel_event);

    // Wait for the kernel to finish
    clWaitForEvents(1, &kernel_event);

    // Query profiling info
    cl_ulong time_start, time_end;
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_START, sizeof(time_start), &time_start, NULL);
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_END, sizeof(time_end), &time_end, NULL);
    double gpu_elapsed_ms = (time_end - time_start) * 1e-6;  // Convert nanoseconds to milliseconds
    // --------------------------------------------------------------
    ```
  - We wrap the code below around the host (CPU) function call `copy()`.
    - We call the function `clock()` to record the current number of `ticks` since the program started, and store the result in a variable `cpu_start` of type `clock_t`.
    - Then we call the host (CPU) function `copy()`.
    - Next, we call the function `clock()` again to record the current number of `ticks` again and store the result in a variable `cpu_stop` of type `clock_t`.
    - Finally, we calculate the elapsed number of milliseconds as `float cpu_elapsed_ms = (double)(cpu_stop - cpu_start) / CLOCKS_PER_SEC * 1000.0`.
  
    ```c
    // --------------------------------------------------------------
    // Timing the host (CPU) function execution time
    // --------------------------------------------------------------
    clock_t cpu_start = clock();

    // Host function call
    copy(h_input, h_output_cpu, N);
    
    clock_t cpu_stop = clock();
    float cpu_elapsed_ms = (double)(cpu_stop - cpu_start) / CLOCKS_PER_SEC * 1000.0;
    // --------------------------------------------------------------
    ```
  - We print out the execution time for the device (GPU) kernel and host (GPU) function.

    ```c
    printf("GPU execution time  : %f ms\n", gpu_elapsed_ms);
    printf("CPU execution time  : %f ms\n", cpu_elapsed_ms);
    ```
  - We use a separate `int` pointer variable `h_output_cpu` for storing the output from the host (CPU) function call.
    - We verify the output results from the device (GPU) kernel and host (CPU) function are the same.
    - This is a common best practice when verifying the correct functionality of an algorithm implemented in a device (GPU) kernel.
      - We use the `abs()` function to compute the absolute difference between each eleement pair in the two arrays.
      - If we were using `float`s instead of `int`s, we can use the `fabs()` function and compare the difference to e.g. `1e-5`.

    ```c
    int errorsum = 0;
    for (int i = 0; i < N; i++)
    {
        int error = abs(h_output[i] - h_output_cpu[i]);
        if (error > 0)
        {
            //printf("Result verification failed for element with index %d!\n", i);
            errorsum += error;
        }
    }
    // Print verification result
    printf("\nVerification : %s\n", (errorsum > 0) ? "FAILED" : "PASSED");
    ```
  - We also print out the two arrays as before (same code) after launching the device (GPU) kernel and after calling the host (CPU) function.
  - Lastly, we also free all memory, including the OpenCL event.

    ```c
    clReleaseEvent(kernel_event)
    ```
- Run the cells below to see the output.
  - We won't record time in this notebook going forward (where possible) to make the example code clearer, but now you know how to do it yourself-

In [41]:
%%writefile src/kernel.cl
__kernel void mykernel(__global const int *input, __global int *output, int n)
{
    int idx = get_global_id(0);
    output[idx] = input[idx];
}

Overwriting src/kernel.cl


In [42]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "utils.h"

#define N 5
#define WORKITEMS_PER_WORKGROUP_0 2

// Host function
void copy(int *input, int *output, int n)
{
    for(int idx = 0; idx < n; idx++)
    {
        output[idx] = input[idx];
    }
}

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    srand(0);
    
    int *h_input, *h_output, *h_output_cpu;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);
    h_output_cpu = (int *)malloc(data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    // Device buffers
    cl_mem d_input, d_output;
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, N * sizeof(int), h_input, &err); // CL_MEM_COPY_HOST_PTR copies h_input values to device buffer
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, N * sizeof(int), NULL, &err);

    // Set kernel arguments
    cl_int n = N;
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    clSetKernelArg(kernel, 2, sizeof(int), &n);

    // Kernel launch configuration
    size_t localSize = WORKITEMS_PER_WORKGROUP_0;
    size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0;

    // --------------------------------------------------------------
    // Timing the device (GPU) kernel execution time
    // --------------------------------------------------------------
    // Enqueue kernel with event
    cl_event kernel_event;
    clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, &kernel_event);

    // Wait for the kernel to finish
    clWaitForEvents(1, &kernel_event);

    // Query profiling info
    cl_ulong time_start, time_end;
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_START, sizeof(time_start), &time_start, NULL);
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_END, sizeof(time_end), &time_end, NULL);
    double gpu_elapsed_ms = (time_end - time_start) * 1e-6;  // Convert nanoseconds to milliseconds
    // --------------------------------------------------------------

    // Read result back (CL_TRUE makes this a synchronous (blocking) call)
    clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, N * sizeof(int), h_output, 0, NULL, NULL);
    
    // Print measured device kernel execution time
    printf("GPU execution time  : %f ms\n", gpu_elapsed_ms);

    // Print elements in both arrays
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    printf("\n");

    // --------------------------------------------------------------
    // Timing the host (CPU) function execution time
    // --------------------------------------------------------------
    clock_t cpu_start = clock();

    // Host function call
    copy(h_input, h_output_cpu, N);
    
    clock_t cpu_stop = clock();
    float cpu_elapsed_ms = (double)(cpu_stop - cpu_start) / CLOCKS_PER_SEC * 1000.0;
    // --------------------------------------------------------------

    // Print measured host function execution time
    printf("CPU execution time  : %f ms\n", cpu_elapsed_ms);

    // Print elements in both arrays
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output_cpu[i]);
    }

    // Verify the results in the GPU output with the CPU output
    int errorsum = 0;
    for (int i = 0; i < N; i++)
    {
        int error = abs(h_output[i] - h_output_cpu[i]);
        if (error > 0)
        {
            //printf("Result verification failed for element with index %d!\n", i);
            errorsum += error;
        }
    }
    
    // Print verification result
    printf("\nVerification : %s\n", (errorsum > 0) ? "FAILED" : "PASSED");

    // Cleanup
    free(h_input);
    free(h_output);
    free(h_output_cpu);

    clReleaseMemObject(d_input);
    clReleaseMemObject(d_output);
    clReleaseEvent(kernel_event); // Release the event

    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [43]:
!{build_multi_file_command}
!{execute_command}

GPU execution time  : 0.010240 ms

input   output
83      83    
86      86    
77      77    
15      15    
93      93    

CPU execution time  : 0.001000 ms

input   output
83      83    
86      86    
77      77    
15      15    
93      93    

Verification : PASSED


### Inspecting the Output

- In the output we see that the execution time on the GPU is slower than on the CPU.
- This is expected since copying 5 elements from one array to another is just a waste of time on a GPU.
  - **Not all problems are suitable for a GPU, in which case we should use the CPU instead**.
- We also see the results verification `PASSED` so we can rest assured that the kernel function is correct (if the CPU function is correct, of course).

---
## 2.7 Shared Memory and Thread Synchronization on the Device (GPU)

- `Shared memory`, called `Local memory` in OpenCL, is a fast, low-latency memory located on-chip, accessible by all `work items` (threads) in a `work group`.
  - **Location**: On-chip, accessible by all `work items` (threads) in a `work group`.
  - **Access**: Readable and writeable by all `work items` (threads) in a `work group` (also `writable` from the `host` before kernel launch).
  - **Size limit**: Typically `48 KB` per `CU` (Compute Unit).
  - **Speed**: Very fast, much faster than `global memory`.
  - **Scope**: Shared only among `work items` (threads) in the same `work group`.
  - **Lifetime**: Exists for the duration of the `work group`.
- Use `shared memory` (`local memory`) when:
  - Work items (threads) need to cooperate, such as tiling, caching, or communication between `work items` (threads).
    - `Shared memory` (`local memory`) is specified within a kernel function with the qualifier `__local`, or as a kernel function parameter.
    - It can be initialized `statically` (inside the kernel function) or `dynamically` (in a kernel function parameter).
- `Work item (thread) synchronization` is used to synchronize `work items` (threads), especially `workitems` (threads) in a `work group` when using `shared memory` (`local memory`):
  - Purpose: Barrier synchronization — all `work items` (threads) in the `work group` must reach it before any can proceed.
    - A barrier (work item/thread synchronization) is done with the command `barrier(CLK_LOCAL_MEM_FENCE)` in a kernel function.
  - Ensures all `shared memory` (`local memory`) reads/writes are complete before continuing.
  - Used to prevent `race conditions`.
- Issues to be aware of when using `shared memory` (`local memory`):
  - `Non-coalesced memory access` also applies when accessing `global memory` (actually more important in that case).
  - `Wavefront (thread) divergence` applies to `work items` (threads) within the same `wavefront` (a `wavefront` is a group of `32` work items (threads) that are scheduled to run on `PEs` within a `CU`/`work group`).
  - `Low occupancy` is related to `shared memory` (`local memory`) but applies more generally to a `kernel launch`.

  | Issue                       | Consequence                        | Fix                                |
  | --------------------------- | ---------------------------------- | ---------------------------------- |
  | Race conditions             | Wrong results                      | Use `barrier(CLK_LOCAL_MEM_FENCE)` or atomics   |
  | No synchronization          | Inconsistent reads/writes          | Use `barrier(CLK_LOCAL_MEM_FENCE)`              |
  | [Bank conflicts](https://www.youtube.com/watch?v=CZgM3DEBplE)              | Performance slowdown               | Pad arrays, restructure access     |
  | Exceeding memory limit      | Kernel launch fails or runs slower | Reduce usage, use fewer threads    |
  | Wrong indexing              | Wrong data or crash                | Use `get_local_id()` properly           |
  | Uninitialized/out-of-bounds | Undefined behavior                 | Always initialize and guard bounds |
  | [Non-coalesced memory access](https://www.youtube.com/watch?v=mLxZyWOI340&list=PLAwxTw4SYaPnFKojVQrmyOGFCqHTxfdv2&index=97)| Slower execution speed | Coalesce memory access |
  | [Wavefront (thread) divergence](https://www.youtube.com/watch?v=bHkFV-YMxxY&list=PLAwxTw4SYaPnFKojVQrmyOGFCqHTxfdv2&index=106) | Slower execution speed | Avoid branches and loops |
  | [Low occupancy](https://www.youtube.com/watch?v=2NGQvnT_3gU) | Slower execution speed | Increase occupancy |  

<br />  

- Now, let's look at a simple example of using `shared memory` (called `local memory` in OpenCL).
- The code is the same as before, but with the following modifications:
  - In the `kernel` function, we declare a buffer (array) with the `__local` qualifier.
    - The `__local` qualifier means the variable/parameter is stored in `shared memory` (`local memory`).
  - The `shared memory` (`local memory`) can be declared with a `static` size or with a `dynamic size`.
  - In the sample code, we are using a `dynamic size`, where
    - the size of the buffer (shared/local memory) is set in the `main()` function.
    - the buffer is passed in as an argument to the kernel function (final parameter in our code), i.e. `__local int *shared`.
  - If we wanted a `static` size, we would:
    - remove the final parameter in the kernel's function header.
    - define the size e.g. using a symbolic constant, e.g. `#define WORKITEMS_PER_WORKGROUP_0 2`
    - uncomment the first row in the kernel function's body in our code below.

  ```c
  //#define WORKITEMS_PER_WORKGROUP_0 2 // when using static local (shared) memory size
  
  //__kernel void mykernel(__global const int *input, __global int *output, const int n)                    // when using static local (shared) memory size
  __kernel void mykernel(__global const int *input, __global int *output, const int n, __local int *shared) // dynamic local (shared) memory size
  {
      //__local int shared[WORKITEMS_PER_WORKGROUP_0]; // when using static local (shared) memory size
  }
  ```
- Let's look at the complete kernel function:
  - We decalare `shared memory` (`local memory`) with a dynamic size via the kernel function's final parameter, i.e. `__local int *shared`
  - Then we get a `work item`'s (thread's) global index/ID (`g_idx`) and a `work item`'s (thread's) local/shared index/ID (`s_idx`).
    - We have to be careful in how we use the work items (threads) for indexing (`g_idx` is unique within a kernel launch, `s_id` is unique within a `work group` on the same `CU`).
    - Remember, if we have `get_local_size(0)` `work items` (threads) per `work group` then `s_id` ranges from `0` to `get_local_size(0) - 1`.
  - Our usual `boundary guard` comes next `if(g_idx >= n)`.
  - Then we copy elements from the `input` array into `shared` (`local`) memory.
    - The index into the `input` array is `g_idx`.
    - The index into the `shared` array is `s_idx`.
    - Different indexing schemes might ne necessary depending on the problem/algorithm.
  - Next, we have a work item (thread) barrier `barrier(CLK_LOCAL_MEM_FENCE)`.
    - This ensures no `work item` (thread) within the `work group` can continue past this row until all `work items` (threads) in the `work group` have completed the code above this row.
      - This is important, since some `work items` (threads) might not have copied their element from the `input` array into the `shared` array yet.
      - In this example, it isn't an issue, because no other `work item` (thread) will read another `work item`'s (thread's) element in the `shared` array in the code below the barrier `barrier(CLK_LOCAL_MEM_FENCE)`.
      - For other problems, this might not be the case, so if `work items` (threads) aren't synchronized, they might continue and read stale data from the `shared` array.
  - Lastly, when all `work items` (threads) are synchronized, a `work item` (thread) copies an element from the `shared` array into the `output` array.
    - The index into the `output` array is `g_idx`.
    - The index into the `shared` array is `s_idx`.
    - Different indexing schemes might ne necessary depending on the problem/algorithm.

  ```C
  //#define WORKITEMS_PER_WORKGROUP_0 2 // when using static local (shared) memory size

  //__kernel void mykernel(__global const int *input, __global int *output, const int n)                    // when using static local (shared) memory size
  __kernel void mykernel(__global const int *input, __global int *output, const int n, __local int *shared) // dynamic local (shared) memory size
  {
      //__local int shared[WORKITEMS_PER_WORKGROUP_0]; // when using static local (shared) memory size

      int g_idx = get_global_id(0); // index in global memory (globally unique)
      int s_idx = get_local_id(0);  // index in local (shared) memory (unique within a workgroup)

      if (g_idx >= n) return; // boundary guard

      // Copy elements in global memory (input) to local (shared) memory
      shared[s_idx] = input[g_idx];

      // Synchronize workitems (threads)
      barrier(CLK_LOCAL_MEM_FENCE); // all workitems (threads) in the same workgroup must be done with the operations above before any workitem (thread) can continue

      // Copy elements in local (shared) memory to global memory (output)
      output[g_idx] = shared[s_idx];
  }
  ```
- Now, let's look at modifications in the `main()` function (most of the code is the same as before, but with the timing removed for clarity).
  - In fact, there is only one modification:
    - Since we are using a dynamic size for our `shared memory` (`local memory`), we first define the size of the memory with `#define WORKITEMS_PER_WORKGROUP_0 2`.
    - Then pass the size of the `shared memory` (`local memory`) (in bytes) as the last argument to the kernel function using the function `clSetKernelArg`.
    - If we were using a static size, we would just remove the final parameter from the kernel function, and remove this `clSetKernelArg` row below.
  - Best practice is to use a dynamic size, since we can determine a variable size in the code.

    ```c
    #define WORKITEMS_PER_WORKGROUP_0 2
    clSetKernelArg(kernel, 3, WORKITEMS_PER_WORKGROUP_0 * sizeof(int), NULL); // dynamic local (shared) memory size (remove when using static size)
    ```
- Run the cells below to see the output (which is exactly the same as before).

In [44]:
%%writefile src/kernel.cl
//#define WORKITEMS_PER_WORKGROUP_0 2 // when using static local (shared) memory size

//__kernel void mykernel(__global const int *input, __global int *output, const int n)                    // when using static local (shared) memory size
__kernel void mykernel(__global const int *input, __global int *output, const int n, __local int *shared) // dynamic local (shared) memory size
{
    //__local int shared[WORKITEMS_PER_WORKGROUP_0]; // when using static local (shared) memory size

    int g_idx = get_global_id(0); // index in global memory (globally unique)
    int s_idx = get_local_id(0);  // index in local (shared) memory (unique within a workgroup)

    if (g_idx >= n) return; // boundary guard

    // Copy elements in global memory (input) to local (shared) memory
    shared[s_idx] = input[g_idx];

    // Synchronize workitems (threads)
    barrier(CLK_LOCAL_MEM_FENCE); // all workitems (threads) in the same workgroup must be done with the operations above before any workitem (thread) can continue

    // Copy elements in local (shared) memory to global memory (output)
    output[g_idx] = shared[s_idx];
}

Overwriting src/kernel.cl


In [45]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "utils.h"

#define N 5
#define WORKITEMS_PER_WORKGROUP_0 2

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    srand(0);
    
    int *h_input, *h_output;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    // Device buffers
    cl_mem d_input, d_output;
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, data_size, h_input, &err);
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, data_size, NULL, &err);    

    // Set kernel arguments
    cl_int n = N;
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    clSetKernelArg(kernel, 2, sizeof(int), &n);
    clSetKernelArg(kernel, 3, WORKITEMS_PER_WORKGROUP_0 * sizeof(int), NULL); // dynamic local (shared) memory size (remove when using static)

    // Kernel launch configuration
    size_t localSize = WORKITEMS_PER_WORKGROUP_0;
    size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0;

    // Enqueue kernel
    clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, NULL);

    // Read result back
    clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, data_size, h_output, 0, NULL, NULL);

    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    printf("\n");

    // Cleanup
    free(h_input);
    free(h_output);
    clReleaseMemObject(d_input);
    clReleaseMemObject(d_output);
    
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [46]:
!{build_multi_file_command}
!{execute_command}


input   output
83      83    
86      86    
77      77    
15      15    
93      93    



### Inspecting the Output

- The output is exactly the same as before (same algorithm, just using different type of memory).

---
## 2.8 Constant Memory on the Device (GPU)

- `Constant memory` is a special type of GPU memory optimized for cases where many `work items` (threads) read the same values.
  - **Location**: On-device, separate from `global memory`.
  - **Access**: Readable by all `work items` (threads) and is `read-only` from the `device`, but `writable` from the `host`.
  - **Size limit**: `64 KB` (per `device`).
  - **Speed**: Very fast if all `work items` (threads) access the same address.
  - **Scope**: Globally accessible (like global variables).
  - **Lifetime**: Exists for the duration of the `kernel launch`.
- Use `constant memory` when:
  - All or most `work items` (threads) access the same data (e.g., coefficients, transformation matrices, filters).
  - The data is known before kernel launch and doesn't change during execution.
  - The data is small (<= `64 KB`).
- Let's look at a simple example using `constant memory`.
- It's the same code as before, but with the `shared memory` (`local memory`) removed, and with the following modifications:
  - In the kernel function:
    - We have replaced the final parameter from using `__local` (shared) memory to using `__constant` memory.
      - It is declared as `__constant const int *coefficients`.
      - The `__constant` qualifier makes this `constant memory`.
    - Then we multiply an element in the `input` vector with an element in `constant` memory with the same index, and store the product in the `output` array.
      - In the `main()` function, we initialize all the `constant` memory elements to a value of `2`.
      - So, compared to the previous code, the `output` vector's values will be twice as large as the `input` vector's values.
    - Note that we have declared the size of the `constant memory` to be the same as the number of elements `N`.
      - This is perfectly fine for this example where `N = 5`, but `constant memory` is extremely limited (small).
      - We wouldn't be able to use `N` as the `constant memory`'s size if, say, `N` was `1000000` (a million elements).

      ```c
      __kernel void mykernel(__global const int *input, __global int *output, const int n, __constant const int *coefficients) // constant memory
      {
          int idx = get_global_id(0); // index in global memory (globally unique)

          if (idx >= n) return; // boundary guard

          // Multiply input elements with coefficients in constant memory and store the product in output
          output[idx] = input[idx] * coefficients[idx];
      }
      ```
  - In the main() function, the code is the same as before, but with `shared memory` removed, and with the following modifications:
    - We declare an `int` pointer variable on the host (CPU) to define the contents to be copied to the `constant memory`.

      ```c
      int *h_coefficients;
      ```
    - We create a variable with the same size (but in bytes) as the statically defined `constant memory`.

      ```c
      int constant_size = N * sizeof(int);
      ```
    - We allocate space in host (CPU) memory (RAM) for the data we will be copying to the `constant memory`.

      ```c
      h_coefficients = (int *)malloc(constant_size);
      ```
    - We initialize the data we will be copying tp `constant memory`.
      - Notice, all the elements in `h_coefficients` are `2` (so the elements in the `output` array from the kernel function will be twice as large as in the `input` array).

      ```c
      for(int i = 0; i<N; i++)
      {
         h_coefficients[i] = 2;
      } 
      ```
    - Then we copy the host (CPU) memory to the device (GPU) `constant memory` using the OpenCL function `clCreateBuffer` as before.
      - We pass the `context` object as the first argument.
      - We pass the flag `CL_MEM_READ_ONLY` bitwise or:ed with the flag `CL_MEM_COPY_HOST_PTR` as the second argument (allocate and initialize read-only memory).
      - We pass the size (in bytes) of the `constant` memory as the third argument.
      - We pass in a pointer to the host (CPU) memory `h_coefficients` as the fourth argument.
      - We pass the `error` object as the last argument.
    
      ```c
      cl_mem d_coefficients = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, constant_size, h_coefficients, &err);
      ```
    - At the very end of the `main()` function, we free the memory on the host (CPU) and device (GPU), allocated for the `constant memory`.

      ```c
      free(h_coefficients);
      clReleaseMemObject(d_coefficients);
      ```
- Run the cells below to see the output.

In [47]:
%%writefile src/kernel.cl

__kernel void mykernel(__global const int *input, __global int *output, const int n, __constant const int *coefficients) // constant memory
{
    int idx = get_global_id(0); // index in global memory (globally unique)

    if (idx >= n) return; // boundary guard

    // Multiply input elements with coefficients in constant memory and store the product in output
    output[idx] = input[idx] * coefficients[idx];
}


Overwriting src/kernel.cl


In [48]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "utils.h"

#define N 5
#define WORKITEMS_PER_WORKGROUP_0 2

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    srand(0);
    
    int *h_input, *h_output, *h_coefficients;
    int data_size = N * sizeof(int);
    int constant_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);
    h_coefficients = (int *)malloc(constant_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    for(int i = 0; i<N; i++)
    {
        h_coefficients[i] = 2;
    }

    // Device buffers
    cl_mem d_input, d_output;
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, data_size, h_input, &err);
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, data_size, NULL, &err);    
    cl_mem d_coefficients = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, constant_size, h_coefficients, &err);

    // Set kernel arguments
    cl_int n = N;
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    clSetKernelArg(kernel, 2, sizeof(int), &n);
    clSetKernelArg(kernel, 3, sizeof(cl_mem), &d_coefficients);

    // Kernel launch configuration
    size_t localSize = WORKITEMS_PER_WORKGROUP_0;
    size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0;

    // Enqueue kernel
    clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, NULL);

    // Read result back
    clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, data_size, h_output, 0, NULL, NULL);

    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    printf("\n");

    // Cleanup
    free(h_input);
    free(h_output);
    free(h_coefficients);
    clReleaseMemObject(d_input);
    clReleaseMemObject(d_output);
    clReleaseMemObject(d_coefficients);
    
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [49]:
!{build_multi_file_command}
!{execute_command}


input   output
83      166   
86      172   
77      154   
15      30    
93      186   



### Inspecting the Output

- We see that the values in the `output` array are twice as large compared to the `input` array,

---
# 3. Sample Problems
---

## 3.1 1D Vector Addition on the Host (CPU)

<img src="images/vectoradd_cpu.png" width="500" style="float: right; margin-right: 50px;" />

Let's start with a simple problem.

Problem
  - We have three vectors (arrays) `A`, `B`, and `C`, all with `N` elements each.
  - We want to compute the elementwise sum of `A` and `B`, and store the sum in `C`.

Solution
1. Define number of elements `N=1048576`
2. Create a host function `void vectorAdd(float *A, float *B, float *C, int n)`
    - Loop through vectors `A` and `B` with `idx=0..N-1`
    - Compute `C[idx] = A[idx] + B[idx]`
3. Create a host function `main(void)`
    - Declare and allocate memory for vectors `h_A`, `h_B`, and `h_C`.
    - Initialize vectors `h_A` and `h_B` with `N` random floats each.
    - Call function `vectorAdd` with `h_A`, `h_B`, `h_C`, `N`, and measure the execution time for `vectorAdd`.
    - Verify result is correct.
    - Print execution time, verification result, and sample elements in vectors `h_A`, `h_B`, and `h_C`.
    - Free memory allocated for vectors `h_A`, `h_B`, and `h_C`.

In [50]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

// Number of elements (1048576)
#define N (1 << 20) 

// Host function (elementwise addition of vectors A and B, placing the sum in vector C)
void vectorAdd(float *A, float *B, float *C, int n)
{   
    // Loop through vectors and compute sum C = A + B
    for (int idx = 0; idx < n; idx++)
    {
        C[idx] = A[idx] + B[idx];
    }
}

// Host main routine
int main(void)
{
    // Seed pseudorandom number generator
    srand(0);

    // Compute the size of the vectors (in bytes)
    size_t size = N * sizeof(float);

    // Declare and allocate host vectors A, B, and C   
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize host input vectors A and B with random values between 0 and 1.0
    for (int i = 0; i < N; ++i)
    {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    // Call function vectorAdd with timing
    clock_t start = clock();

    vectorAdd(h_A, h_B, h_C, N); // function call
    
    clock_t end = clock();
    double elapsed_ms = (double)(end - start) / CLOCKS_PER_SEC * 1000.0;
    
    // Verify results in ouput vector C is correct
    float errorsum = 0.0f;
    for (int i = 0; i < N; ++i)
    {
        float error = fabs(h_A[i] + h_B[i] - h_C[i]);
        if (error > 1e-5)
        {
            //printf("Result verification failed for element with index %d!\n", i);
            errorsum += error;
        }
    }

    // Print measured function execution time, verification result, and sample elements from each vector
    printf("CPU execution time  : %f ms\n", elapsed_ms);
    printf("Verification result : %s\n", (errorsum > 1e-5) ? "FAILED" : "PASSED");
    printf("Vector samples      : A[0]=%f, B[0]=%f, C[0]=%f\n", h_A[0], h_B[0], h_C[0]);
    
    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Overwriting src/main.c


In [51]:
!{build_multi_file_command}
!{execute_command}

CPU execution time  : 2.252000 ms
Verification result : PASSED
Vector samples      : A[0]=0.840188, B[0]=0.394383, C[0]=1.234571


---
## 3.2 1D Vector Addition on the Device (GPU)

<img src="images/vectoradd_gpu1.png" width="450" style="float: right; margin-right: 50px;" />

Problem
  - We have three vectors (arrays) `A`, `B`, and `C`, all with `N` elements each.
  - We want to compute the elementwise sum of `A` and `B`, and store the sum in `C`.

We Know
- In OpenCL, we have access to many `work items` (threads), where `work items` (threads) are organized into `work groups`, and `work groups` are organized into an NDRange`.
  - `get_local_id(0)` represents a `work item’s index` along the `0` dimension within a `work group`.
  - `get_group_id(0)` represents a `work group’s index` along the `0` dimension within the `NDRange`.
  - `get_group_size(0)` represents the `number of work items` along the `0` dimension with a `work group`.
  - `get_global_id(0)` represents a `work item`'s (thread's) global ID (`index`) on the device.
- `Work groups` are assigned to a Compute Unit (CU) that has a number of Processing Elements (PEs).
  - Each `work item` (thread) executes its own copy of the `kernel function`, in parallel, with the same parameter values.
  - Each `work item` (thread) should process only one element in the arrays using the `index`.
  - If there are more work items (threads) than elements (`index >= N`), those work items (threads) should `return` immediately from the `kernel function`
- There can be a maximum of `1024` work items (threads) in a block.
  - If we have `N = 1048576` elements, and `WORKITEMS_PER_WORKGROUP_0 = 1024`, we get:
  - `local_size = WORKITEMS_PER_WORKGROUP_0 = 1024`.
  - `global_size = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0 = ((1048576+1024-1) / 1024) * 1024 = 1024 * 1024 = 1048576`.
  - `group_size = global_size / local_size = 1024`.
  - And if we have `24` CUs, each will be assigned roughly `1024 / 24 = 42` work groups for maximum efficiency.

<img src="images/vectoradd_gpu2.png" width="450" style="float: right; margin-right: 50px;" />

Solution
1. Define number of elements `N=1048576` and `WORKITEMS_PER_WORKGROUP_0=1024`
2. Create a kernel `__kernel void mykernel(__global const float *A, __global const float *B, __global float *C, const int n)`
    - Get global work item (thread) ID `idx = get_global_id(0)`
    - Return if index is out of bounds (`idx >= n`) which means we have more work items (threads) than elements `n`.
      - In this case we won't since `N` is evenly divisible by `WORKITEMS_PER_WORKGROUP_0`.
    - Compute `C[idx] = A[idx] + B[idx]`.

3. Create a host function `main(void)`
    - Declare and allocate memory for host vectors `h_A`, `h_B`, and `h_C`.
    - Initialize host vectors `h_A` and `h_B` with `N` random floats each.
    - Declare and allocate memory for device vectors `d_A`, `d_B`, and `d_C`.
      - Copy contents of host vectors `h_A` and `h_B` to device vectors `d_A` and `d_B`.
    - Set kernel arguments for `d_A`, `d_B`, `d_C`, and `N`.
    - Set launch parameters size_t `localSize` and `globalSize` as described above.
    - Enqueue kernel with launch configuration in `localSize` and `globalSize`.
    - Copy contents of device vector `d_C` to host vector `h_C`.
    - Verify result is correct.
    - Print execution time, verification result, and sample elements in host vectors `h_A`, `h_B`, and `h_C`.
    - Free memory allocated for device vectors `d_A`, `d_B`, and `d_C`.
    - Free memory allocated for host vectors `h_A`, `h_B`, and `h_C`.

<img src="images/coalesced_memory_access.png" width="450" style="float: right; margin-right: 50px;" />

No need for shared (local) or constant memory, and the global memory access pattern is **coalesced** in the code, (a) in the figure.

In [52]:
%%writefile src/kernel.cl
// Device kernel (elementwise addition of vectors A and B, placing the sum in vector C)
__kernel void mykernel(
    __global const float *A,
    __global const float *B,
    __global float *C,
    const int n)
{
    // Compute index (idx) from global workitem (thread) ID
    int idx = get_global_id(0);

    // Return if index is out of bounds (means we have more workitems (threads) than elements)
    if (idx >= n)
        return;

    // Compute the sum C = A + B for the element with index idx
    C[idx] = A[idx] + B[idx];
}

Overwriting src/kernel.cl


In [53]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include "utils.h"

// Number of elements (1048576)
#define N (1 << 20)

// Number of threads
#define WORKITEMS_PER_WORKGROUP_0 1024

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    // Seed pseudorandom number generator
    srand(0);
    
    // Compute the size of the vectors (in bytes)
    size_t size = N * sizeof(float);

    // Declare and allocate host vectors A, B, and C   
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize host input vectors A and B with random values between 0 and 1.0
    for (int i = 0; i < N; ++i)
    {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    // Allocate the device input vectors A, B, and copy data from host vectors A, B
    // Allocate the device output vector C
    cl_mem d_A, d_B, d_C;
    d_A = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, size, h_A, &err);
    d_B = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, size, h_B, &err);
    d_C = clCreateBuffer(context, CL_MEM_WRITE_ONLY, size, NULL, &err);

    // Set kernel arguments
    cl_int n = N;
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_A);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_B);
    clSetKernelArg(kernel, 2, sizeof(cl_mem), &d_C);
    clSetKernelArg(kernel, 3, sizeof(int), &n);

    // Kernel launch configuration
    size_t localSize = WORKITEMS_PER_WORKGROUP_0;
    size_t globalSize = ((N + WORKITEMS_PER_WORKGROUP_0 - 1) / WORKITEMS_PER_WORKGROUP_0) * WORKITEMS_PER_WORKGROUP_0;

    // Enqueue kernel with timing event
    cl_event kernel_event;
    clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, &kernel_event);
    
    // Wait for kernel to finish and compute execution time
    clWaitForEvents(1, &kernel_event);
    cl_ulong time_start, time_end;
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_START, sizeof(time_start), &time_start, NULL);
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_END, sizeof(time_end), &time_end, NULL);
    double elapsed_ms = (time_end - time_start) * 1e-6;  // Convert nanoseconds to milliseconds

    // Read result back from device output vector C to host output vector C
    clEnqueueReadBuffer(queue, d_C, CL_TRUE, 0, size, h_C, 0, NULL, NULL);

    // Verify the result vector is correct
    float errorsum = 0.0f;
    for (int i = 0; i < N; ++i)
    {
        float error = fabs(h_A[i] + h_B[i] - h_C[i]);
        if (error > 1e-5)
        {
            //fprintf(stderr, "Result verification failed at element %d!\n", i);
            errorsum += error;
        }
    }
  
    // Print measured kernel execution time, verification result, and sample elements from each vector
    printf("GPU execution time  : %f ms\n", elapsed_ms);
    printf("Verification result : %s\n", (errorsum > 1e-5) ? "FAILED" : "PASSED");
    printf("Vector samples      : A[0]=%f, B[0]=%f, C[0]=%f\n", h_A[0], h_B[0], h_C[0]);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    // Free device global memory and event
    clReleaseMemObject(d_A);
    clReleaseMemObject(d_B);
    clReleaseMemObject(d_C);
    clReleaseEvent(kernel_event);
    
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [54]:
!{build_multi_file_command}
!{execute_command}

GPU execution time  : 0.033792 ms
Verification result : PASSED
Vector samples      : A[0]=0.840188, B[0]=0.394383, C[0]=1.234571


---
## 3.3 1D Convolution on the Host (CPU)

<img src="images/1dconvolution.gif" width="600" style="float: right; margin-right: 50px;" />

Next, let's tackle the problem of a 1-dimensional (1D) convolution.

Problem
- We have an `input` vector, a `kernel` (filter), and an `output` vector.
- We want to slide the `kernel` (filter) over each element in the `input` vector.
- The `kernel` (filter) will be centered over each element in the `input` vector.
- So the `kernel`'s (filter's) width has to be odd, e.g. `1x3`, `1x5`, `1x7`.
- We multiply each element under the `kernel` (filter) in the `input` vector with `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` vector with the same `index` as the current `input` vector.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` vector, we use `zero-padding`.

Solution
1. Define number of elements `N=1048576`
2. Create a function:
   - `void convolve1D(float *input, float *output, float *filter)`
   - Loop through `input` vector.
   - Compute `output[idx] = input[idx + offset] = filter[FILTER_WIDTH/2 + offset]`
     - Only if `if(idx + offset >= 0 && idx + offset < DATA_WIDTH)`
     - Where `offset` ranges from `-FILTER_WIDTH/2` to `+FILTER_WIDTH/2`.
   - This computation is equivalent to
     - Looping through the `input` vector, zero-padded with `FILTER_WIDTH/2` elements on both sides.
     - Centering the `filter` over each original element in the zero-padded `input` vector.
     - Computing the weighted sum and storing it in the `output` vector.
4. Create a function `main(void)`
   - Define a `DATA_WIDTH`, `FILTER_WIDTH` and `FILTER_WIDTH_OFFSET` (which is `FILTER_WIDTH/2`).
   - Declare and allocate memory for vectors `input`, `ouput`, and `filter`.
   - Initialize vector `input` with `DATA_WIDTH` random floats.
   - Initialize vector `filter` with `weights` where each weight is `1.0 / FILTER_WIDTH` (averaging filter).
   - Call function `convolve1D` with `input`, `ouput`, and `filter`.
   - Measure the execution time for `convolve1D`.
   - Print execution time and sample elements in vectors `input` and `output`.
   - Free memory allocated for vectors `input`, `output`, and `filter`.

In [55]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

// Number of data elements (1048576)
#define DATA_WIDTH (1 << 20)

// Number of filter elements 
#define FILTER_WIDTH 3

// Number of elements on each side of a centered filter
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH / 2)

void convolve1D(float *input, float *output, float *filter)
{
    // Loop through all elements
    for(int d_col = 0; d_col < DATA_WIDTH; d_col++)
    {
        // Apply filter (slide filter over data and compute weighted sum)
        float sum = 0.0f;
        for (int offset_col = -FILTER_WIDTH_OFFSET; offset_col <= FILTER_WIDTH_OFFSET; offset_col++)
        {
            int f_col = FILTER_WIDTH_OFFSET + offset_col; // f_col: 0..FILTER_WIDTH-1
            int i_col = d_col + offset_col;               // i_col: 0-FILTER_WIDTH_OFFSET..DATA_WIDTH-1+FILTER_WIDTH_OFFSET
            
            if(i_col >= 0 && i_col < DATA_WIDTH)
            {
                sum += input[i_col] * filter[f_col];
            }
        }
        
        // Store the weighted sum in the output array
        output[d_col] = sum;
    }
}

int main(void)
{
    // Seed the random number generator
    srand(0);            // use this for same set of random numbers each time the program is run
    //srand(time(NULL)); // use this for different set of random numbers each time the program is run

    // Declare variables
    float *h_input, *h_output, *h_filter; // host copies of input, output, filter
    int data_size = DATA_WIDTH * sizeof(float);     // size of data in bytes
    int filter_size = FILTER_WIDTH * sizeof(float); // size of filter in bytes
   
    // Allocate space for host copies of input, output, filter
    h_input = (float *)malloc(data_size);
    h_output = (float *)malloc(data_size);
    h_filter = (float *)malloc(filter_size);
      
    // Setup input values
    for (int col = 0; col < DATA_WIDTH; col++)
    {
        h_input[col] = (float)rand() / RAND_MAX; // Random floats between 0 and 1.0
    }

    // Setup filter
    for (int col = 0; col < FILTER_WIDTH; col++)
    {
        h_filter[col] = 1.0f / FILTER_WIDTH; // averaging filter
    }
   
    // Call convolve1D() with timing
    clock_t start = clock();                                              // record the start time
    convolve1D(h_input, h_output, h_filter);                              // call convolve1D()
    clock_t stop = clock();                                               // record the stop time
    double elapsed_ms = (double)(stop - start) / CLOCKS_PER_SEC * 1000.0; // calculate the elapsed time in millisecond

    // Print measured calculation execution time
    printf("Calculation (%d elements, 1x%d filter) took %.2f ms\n", DATA_WIDTH, FILTER_WIDTH, elapsed_ms);
   
    // Print out the FILTER_WIDTH number of elements in the two arrays
    printf("Vector samples:\n");
    for(int i = 0; i < FILTER_WIDTH; i++)
    {
        printf("h_input[%d]=%.2f, h_output[%d]=%.2f\n", i, h_input[i], i, h_output[i]);
    }

    // Cleanup
    free(h_input);
    free(h_output);
    free(h_filter);
    
    return 0;
 }

Overwriting src/main.c


In [56]:
!{build_multi_file_command}
!{execute_command}

Calculation (1048576 elements, 1x3 filter) took 7.04 ms
Vector samples:
h_input[0]=0.84, h_output[0]=0.41
h_input[1]=0.39, h_output[1]=0.67
h_input[2]=0.78, h_output[2]=0.66


- The output shows:
  - Given an element with index `idx` in the `output` array.
  - It's value is the average of the elements with indices `-FILTER_WIDTH_OFFSET..+FILTER_WIDTH_OFFSET+1` in the `input` array.
    - Since an averaging filter was used.
  - For example
    - If the `FILTER_WIDTH` is `3`, we have `FILTER_WIDTH_OFFSET = FILTER_WIDTH / 2 = 1`.
    - The value of an element with index `idx` in the `output` array is the average of the elements with indices `idx-1`, `idx`, and `idx+1` in the `input` array.
      - `output[idx] = (input[idx-1] +  nput[idx] + nput[idx+1]) / 3`
      - If it's a bounday element, the out-of-bounds indices have zero-padded elements with a value of `0`.

---
## 3.4 1D Convolution on the Device (GPU)

<img src="images/tiled_convolution_1d.png" width="600" style="float: right; margin-right: 50px;" />

Problem
- We have an `input` vector, a `kernel` (filter), and an `output` vector.
- We want to slide the `kernel` (filter) over each element in the `input` vector.
- The `kernel` (filter) will be centered over each element in the `input` vector.
- So the `kernel`'s (filter's) width has to be odd, e.g. `1x3`, `1x5`, `1x7`.
- We multiply each element under the `kernel` (filter) in the `input` vector with `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` vector with the same `index` as the current `input` vector.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` vector, we use `zero-padding`.

Solution
- We have a 1D `input` vector with `N` elements (vector marked with `N` in the figure).
- A `work group` of `wotk items` (threads) will process `get_local_size(0)` number of elements (top row in figure).
- We don't want to load elements multiple times from `global memory` during the calulation.
  - So each `work item` (thread) in a `work group` loads its `input` element into `shared memory` (called `Tile` in the figure).
  - The `shared_memory` size needs to be `TILE_BASE_WITH + 2 * FILTER_WIDTH_OFFSET`, where
    - `TILE_BASE_WITH` is the number of original `input` elements in a `work group` (highlighted elements in figure).
    - `FILTER_WIDTH_OFFSET` is `FILTER_WIDTH / 2` (called `halo` elements in the figure).
    - `FILTER_WIDTH` is `5` (in the figure).

<img src="images/block_tile_loading_1d.png" width="400" style="float: right; margin-right: 50px;" />

  - This ensures the `filter`, when centered on an element, covers all neighbouring elements, e.g.
    - In `Work Group 0` the work items (threads) use `Tile 0`, where the original elements are `0`, `1`, `2`, `3` (see figure).
    - The `filter` is centered on `0` covering `FILTER_WIDTH_OFFSET` neighbouring elements on each side.
    - For border elements we use zero-padding (called `ghost` elements in the figure for the left-most elements).
    - So the elements included in the first convolution are `ghost`, `ghost`, `0`, `1`, `2` (where `ghost = 0`).
      - When processing element `3`, the `filter` covers elements `1`, `2`, `3`, `4`, `5`.

- For these extra `2 * FILTER_WIDTH_OFFSET` elements to be available in a `work group`:
    - The `shared memory`, called `Tile`, needs a size of `TILE_BASE_WITH + 2 * FILTER_WIDTH_OFFSET` (see above).
      - This is the actual size need for a `work group` of `work items` (threads), i.e. `get_local_size(0)` which includes:
      - Work items (threads) for loading the original elements that the `filter` will center on.
      - Work items (threads) for the extra `2 * FILTER_WIDTH_OFFSET` border elements.
      - This is illustrated in the bottom figure.
- We also want to load the `filter` elements into `constant` memory to avoid hitting global memory when accessing them.

- So this is what we'll do:
1. Define:
  - `DATA_WIDTH=1048576` (number of elements in data vectors `input` and `output`)
  - `FILTER_WIDTH=3` (number of elements in the `filter` vector)
  - `FILTER_WIDTH_OFFSET=FILTER_WIDTH/2` (number of elements on each size of a centered `filter`)
  - `TILE_WIDTH_BASE=16` (number original elements in a `work group` of `work items` (threads))
    - Where the final tile size is `TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET` to cover border elements.
    - This is also the size we will use for the `shared memory` and `work group` size, i.e. `get_local_size(0)`.
    - So we have these many `work items` (threads) in each `work group` and we know a `work group` is assigned to a `CU`.
2. Define `constant` memory of size `FILTER_WIDTH` for the `filter`.
3. Create a kernel function:
   ```c
   __kernel void mykernel(
       __global const float *input,
       __global float *output,
       __constant const float *filter,
       __local float *shared,
       const int data_width,
       const int filter_width_offset)
    ```
   - Define `shared` memory of size `TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET`.
   - Let the `work items` (threads) in a `work group` load their `input` elements into `shared` (`local`) memory.
     - For border elements, we load the value `0` into `shared` (`local`) memory (zero-padding).
   - Synchronize `work items` (threads) to ensure each `work item` (thread) in a `work group` has loaded its element into `shared` (`local`) memory.
   - Compute the convolution as in the CPU solution, but now using `shared` (`local`) memory (input) and `constant` memory (filter).
   - Store the result in the `ouput` vector.
5. Create a function `main(void)`
   - Declare and allocate memory for vectors `input`, `ouput`, and `filter` on the host (CPU).
   - Initialize vector `input` with `DATA_WIDTH` random floats on the host (CPU).
   - Initialize vector `filter` with `weights` on the host (CPU).
     - Each weight is `1.0 / FILTER_WIDTH` (averaging filter).
   - Declare and allocate memory for vectors `input` and `ouput`, and `consatnt` memory, on the device (GPU).
     - Copy `input` vector in host (CPU) memory to device (GPU) global memory.
     - Copy `filter` vector in host (CPU) memory to `constant` device (GPU) memory.
   - Set kernel function arguments:

      ```c
      cl_int data_width = DATA_WIDTH;
      cl_int filter_width_offset = FILTER_WIDTH_OFFSET;
      int workgroup_width = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET;
      int shared_size = workgroup_width * sizeof(float);
      clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
      clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
      clSetKernelArg(kernel, 2, sizeof(cl_mem), &d_filter);
      clSetKernelArg(kernel, 3, shared_size, NULL);         // dynamic local (shared) memory size
      clSetKernelArg(kernel, 4, sizeof(int), &data_width);
      clSetKernelArg(kernel, 5, sizeof(int), &filter_width_offset);  
      ```
   - Set kernel launch configuration:

      ```c
      // Kernel launch configuration
      size_t localSize = workgroup_width;
      size_t globalSize = ((DATA_WIDTH + workgroup_width - 1) / workgroup_width) * workgroup_width;
      ```
   - Launch kernel with launch configuration `globalSize` and `localSize`.
   - Measure the execution time for the `kernel function`.
   - Copy `output` vector in device (GPU) memory to host (CPU) memory.
   - Print execution time and sample elements in vectors `input` and `output`.
   - Free memory allocated for vectors `input`, `output`, and `filter` on the host (CPU).
   - Free memory allocated for vectors `input`, `output` and `filter` on the device (GPU).

In [57]:
%%writefile src/kernel.cl
__kernel void mykernel(
    __global const float *input,
    __global float *output,
    __constant const float *filter,
    __local float *shared,
    const int data_width,
    const int filter_width_offset)
{
    int s_col = get_local_id(0);             // Workitem's (thread's) index in shared memory
    int d_col = get_global_id(0);            // Workitem's (thread's) index in global memory
    int i_col = d_col - filter_width_offset; // Workitem's (thread's) offset index in global memory

    // Guard against workitems (threads) with IDs that would index outside the arrays
    if (d_col >= data_width) return;

    // Fill local (shared) memory with elements in global memory
    if (i_col >= 0 && i_col < data_width)
    {
        shared[s_col] = input[i_col];
    }
    else
    {
        shared[s_col] = 0.0f; // zero-padding
    }

    // Make sure each workitem (thread) in the workgroup has entered its element
    // into local (shared) memory before any workitem (thread) continues
    barrier(CLK_LOCAL_MEM_FENCE);

    // Apply filter
    float sum = 0.0f;
    for (int offset_col = -filter_width_offset; offset_col <= filter_width_offset; offset_col++)
    {
        int f_col = filter_width_offset + offset_col;
        int i_col = s_col + f_col;
        
        if(i_col >= 0 && i_col < get_local_size(0))
        {
            sum += shared[i_col] * filter[f_col]; // data elements in local (shared) memory + filter weights in constant memory = super fast computation
        }
    }

    // Store the weighted sum in the output array
    output[d_col] = sum;
}

Overwriting src/kernel.cl


In [58]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include "utils.h"

// Number of data elements (1048576)
#define DATA_WIDTH (1 << 20)

// Number of filter elements
#define FILTER_WIDTH 3

// Number of elements on each side of a centered filter
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH / 2)

// Number of elements in local (shared) memory
#define TILE_WIDTH_BASE 16

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    srand(0);
    //srand(time(NULL));
    
    // Declare variables
    float *h_input, *h_output, *h_filter; // host copies of input, output, filter
    int data_size = DATA_WIDTH * sizeof(float);
    int filter_size = FILTER_WIDTH * sizeof(float);

    // Allocate space for host (CPU) copies of input, output, filter
    h_input = (float *)malloc(data_size);
    h_output = (float *)malloc(data_size);
    h_filter = (float *)malloc(filter_size);

    // Setup input values
    for (int col = 0; col < DATA_WIDTH; col++)
    {
        h_input[col] = (float)rand() / RAND_MAX; // Random floats between 0 and 1.0
    }

    // Setup filter
    for (int col = 0; col < FILTER_WIDTH; col++)
    {
        h_filter[col] = 1.0f / FILTER_WIDTH; // averaging filter
    }

    // Device buffers
    cl_mem d_input, d_output, d_filter;
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, data_size, h_input, &err);
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, data_size, NULL, &err);    
    d_filter = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, filter_size, h_filter, &err);

    // Set kernel arguments
    cl_int data_width = DATA_WIDTH;
    cl_int filter_width_offset = FILTER_WIDTH_OFFSET;
    int workgroup_width = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET;
    int shared_size = workgroup_width * sizeof(float);
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    clSetKernelArg(kernel, 2, sizeof(cl_mem), &d_filter);
    clSetKernelArg(kernel, 3, shared_size, NULL);         // dynamic local (shared) memory size
    clSetKernelArg(kernel, 4, sizeof(int), &data_width);
    clSetKernelArg(kernel, 5, sizeof(int), &filter_width_offset);  

    // Kernel launch configuration
    size_t localSize = workgroup_width;
    size_t globalSize = ((DATA_WIDTH + workgroup_width - 1) / workgroup_width) * workgroup_width;

    // Enqueue kernel with timing event
    cl_event kernel_event;
    clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, &localSize, 0, NULL, &kernel_event);

    // Wait for kernel to finish and compute execution time
    clWaitForEvents(1, &kernel_event);
    cl_ulong time_start, time_end;
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_START, sizeof(time_start), &time_start, NULL);
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_END, sizeof(time_end), &time_end, NULL);
    double elapsed_ms = (time_end - time_start) * 1e-6;  // Convert nanoseconds to milliseconds

    // Copy result back to host
    clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, data_size, h_output, 0, NULL, NULL);

    // Print measured calculation execution time
    printf("Calculation (%d elements, 1x%d filter) took %.2f ms\n", DATA_WIDTH, FILTER_WIDTH, elapsed_ms);
   
    // Print out the FILTER_WIDTH number of elements in the two arrays
    printf("Vector samples:\n");
    for(int i = 0; i < FILTER_WIDTH; i++)
    {
        printf("h_input[%d]=%.2f, h_output[%d]=%.2f\n", i, h_input[i], i, h_output[i]);
    }

    // Cleanup
    free(h_input);
    free(h_output);
    free(h_filter);
    clReleaseMemObject(d_input);
    clReleaseMemObject(d_output);
    clReleaseMemObject(d_filter);
    clReleaseEvent(kernel_event);
    
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [59]:
!{build_multi_file_command}
!{execute_command}

Calculation (1048576 elements, 1x3 filter) took 0.10 ms
Vector samples:
h_input[0]=0.84, h_output[0]=0.41
h_input[1]=0.39, h_output[1]=0.67
h_input[2]=0.78, h_output[2]=0.66


In the output we see:
- The results are the same for the GPU solution as for the CPU solution.
- The execution time for the GPU solution is significantly fast than the CPU solution.

---
## 3.5 2D Convolution on the Host (CPU)

<img src="images/2dconvolution.gif" width="600" style="float: right; margin-right: 50px;" />

**Note**
  - This is really just the same problem as a 1D convolution, but with an added second dimension.
  - Therefore the problem and solution will be the same, but with the second dimension accounted for.

Problem
- We have an `input` matrix, a `kernel` (filter), and an `output` matrix.
- We want to slide the `kernel` (filter) over each element in the `input` matrix.
- The `kernel` (filter) will be centered over each element in the `input` matrix.
- So the `kernel`'s (filter's) width and height has to be odd, e.g. `3x3`, `5x5`, `7x7`.
- We multiply each element under the `kernel` (filter) in the `input` matrix with the `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` matrix with the same `index` as the current `input` matrix.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` matrix, we use `zero-padding`.

Solution
1. Define:
   - `DATA_WIDTH=32` (number of elements in the `col` dimension for the `input` and `output`)
   - `DATA_HEIGHT=32` (number of elements in the `row` dimension for the `input` and `output`)
   - `FILTER_WIDTH=3` (number of elements int the `col` dimension for the `filter`)
   - `FILTER_HEIGHT=3` (number of elements int the `row` dimension for the `filter`)
   - `FILTER_WIDTH_OFFSET=FILTER_WIDTH/2` (number of elements to the left and right of the centered `filter`)
   - `FILTER_HEIGHT_OFFSET=FILTER_HEIGHT/2` (number of elements above and below the centered `filter`)
3. Create a function:
   - `void convolve2D(float *input, float *output, float *filter)`
   - Loop through `input` matrix.
   - Compute convolution. Store result in `output` matrix.
   - The only difference in the "D convolution compared to the 1D convolution is the additional dimension.
4. Create a function `main(void)`
   - Declare and allocate memory for matrices `input`, `ouput`, and `filter`.
   - Initialize matrix `input` with `DATA_HEIGHT * DATA_WIDTH` random floats.
   - Initialize matrix `filter` with `weights` where each weight is `1.0 / FILTER_HEIGHT * FILTER_WIDTH` (averaging filter).
   - Call function `convolve2D` with `input`, `ouput`, and `filter`.
   - Measure the execution time for `convolve2D`.
   - Print execution time and sample elements in matrices `input` and `output`.
   - Free memory allocated for matrices `input`, `output`, and `filter`.

In [60]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define DATA_WIDTH 32
#define DATA_HEIGHT 32
#define FILTER_WIDTH 3
#define FILTER_HEIGHT 3
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH/2)
#define FILTER_HEIGHT_OFFSET (FILTER_HEIGHT/2)

void convolve2D(float *input, float *output, float *filter)
{
    for(int d_row = 0; d_row < DATA_HEIGHT; d_row++)
    {
        for(int d_col = 0; d_col < DATA_WIDTH; d_col++)
        {
            float sum = 0.0f;
            for (int offset_row = -FILTER_HEIGHT_OFFSET; offset_row <= FILTER_HEIGHT_OFFSET; offset_row++)
            {
                for (int offset_col = -FILTER_WIDTH_OFFSET; offset_col <= FILTER_WIDTH_OFFSET; offset_col++)
                {
                    int f_row = FILTER_HEIGHT_OFFSET + offset_row;
                    int f_col = FILTER_WIDTH_OFFSET + offset_col;
                    int i_row = d_row + offset_row;
                    int i_col = d_col + offset_col;

                    if(i_row >= 0 && i_row < DATA_HEIGHT && i_col >= 0 && i_col < DATA_WIDTH)
                    {
                        sum += input[i_row * DATA_WIDTH + i_col] * filter[f_row * FILTER_WIDTH + f_col];
                    }
                }
            }

            output[d_row * DATA_WIDTH + d_col] = sum;
        }
    }
}

int main(void)
{
    srand(0);

    float *h_input = (float *)malloc(DATA_WIDTH * DATA_HEIGHT * sizeof(float));
    float *h_output = (float *)malloc(DATA_WIDTH * DATA_HEIGHT * sizeof(float));
    float *h_filter = (float *)malloc(FILTER_WIDTH * FILTER_HEIGHT * sizeof(float));

    for(int row = 0; row < DATA_HEIGHT; row++)
    {
        for(int col = 0; col < DATA_WIDTH; col++)
        {
            h_input[row * DATA_WIDTH + col] = (float)rand() / RAND_MAX;
        }
    }

    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            h_filter[row * FILTER_WIDTH + col] = 1.0f / (FILTER_WIDTH * FILTER_HEIGHT);
        }
    }

    // Call convolve2D() with timing
    clock_t start = clock();
    convolve2D(h_input, h_output, h_filter);
    clock_t stop = clock();
    double elapsed_ms = (double)(stop - start) / CLOCKS_PER_SEC * 1000.0;

    printf("Calculation (%d elements, %dx%d filter) took %.2f ms\n", DATA_HEIGHT * DATA_WIDTH, FILTER_HEIGHT, FILTER_WIDTH, elapsed_ms);
    printf("\nMatrix samples:\n");
    printf("h_input %-12s h_output\n", "");
    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_input[row * DATA_WIDTH + col]);
        }
        printf("%-3s","");
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_output[row * DATA_WIDTH + col]);
        }
        printf("\n");
    }

    free(h_input);
    free(h_output);
    free(h_filter);

    return 0;
}

Overwriting src/main.c


In [61]:
!{build_multi_file_command}
!{execute_command}

Calculation (1024 elements, 3x3 filter) took 0.04 ms

Matrix samples:
h_input              h_output
0.840 0.394 0.783    0.238 0.396 0.382 
0.613 0.296 0.638    0.328 0.527 0.568 
0.267 0.540 0.375    0.325 0.514 0.616 


- The output shows:
  - Given an element with index `[row, col]` in the `output` matrix.
  - It's value is the average of the elements with indices:
    - `-FILTER_HEIGHT_OFFSET..+FILTER_HEIGHT_OFFSET+1` in the `input` matrix's `row`.
    - `-FILTER_WIDTH_OFFSET..+FILTER_WIDTH_OFFSET+1` in the `input` matrix's `col`.
    - Since an averaging filter was used.
  - For example
    - If the `FILTER_HEIGHT` is `3`, we have `FILTER_HEIGHT_OFFSET = FILTER_HEIGHT / 2 = 1`.
    - If the `FILTER_WIDTH` is `3`, we have `FILTER_WIDTH_OFFSET = FILTER_WIDTH / 2 = 1`.
    - The value of an element with index `[row, col]` in the `output` matrix is the average of the elements in the `input` matrix with indices:

      ```c
      [row-1, col-1]  [row-1, col]  [row-1, col+1]
      [row  , col-1]  [row  , col]  [row  , col+1]
      [row+1, col-1]  [row+1, col]  [row+1, col+1]
      ```
    - If it's a bounday element, the out-of-bounds indices have zero-padded elements with a value of `0`.

---
## 3.6 2D Convolution on the Device (GPU)

**Note**
  - This is really just the same problem as a 1D convolution, but with an added second dimension.
  - Therefore the problem and solution will be the same, but with the second dimension accounted for.

Problem
- We have an `input` matrix, a `kernel` (filter), and an `output` matrix.
- We want to slide the `kernel` (filter) over each element in the `input` matrix.
- The `kernel` (filter) will be centered over each element in the `input` matrix.
- So the `kernel`'s (filter's) height and width has to be odd, e.g. `3x3`, `5x5`, `7x7`.
- We multiply each element under the `kernel` (filter) in the `input` matrix with the `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` matrix with the same `index` as the current `input` matrix.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` matrix, we use `zero-padding`.

Solution
1. Define:
  -  `DATA_WIDTH=32` (number of elements in the `col` dimension for the `input` and `output`)
   - `DATA_HEIGHT=32` (number of elements in the `row` dimension for the `input` and `output`)
   - `FILTER_WIDTH=3` (number of elements int the `col` dimension for the `filter`)
   - `FILTER_HEIGHT=3` (number of elements int the `row` dimension for the `filter`)
   - `FILTER_WIDTH_OFFSET=FILTER_WIDTH/2` (number of elements to the left and right of the centered `filter`)
   - `FILTER_HEIGHT_OFFSET=FILTER_HEIGHT/2` (number of elements above and below the centered `filter`) 
   - `TILE_WIDTH_BASE=16` (number original elements in a `work group` of `work items` (threads) in the `col` dimension)
   - `TILE_HEIGHT_BASE=16` (number original elements in a `work group` of `work items` (threads) in the `row` dimension)
     - Where the final tile size in the `col` dimension is `TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET` to cover left and right border elements.
       - This is also the size we will use for the `col` dimension in `shared memory` (`local memory`) and `work group` size, i.e. `get_local_size(0)`.
     - Where the final tile size in the `row` dimension is `TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET` to cover top and bottom border elements.
       - This is also the size we will use for the `row` dimension in `shared memory` (`local memory`) and `work group` size, i.e. `get_local_size(1)`.
       - Notice we are now using the second dimension `1`.
     - So we have these many 2D `work items` (threads) in each `work group` and we know a `work group` is assigned to a `CU`.
3. Define a `constant` memory of size `FILTER_HEIGHT * FILTER_WIDTH` for the `filter`.
4. Create a kernel function:

    ```c
    __kernel void mykernel(
        __global const float *input,
        __global float *output,
        __constant const float *filter,
        __local float *shared,
        const int data_height,
        const int data_width,
        const int filter_height_offset,
        const int filter_width_offset,
        const int filter_width)
    ```
   - Define `shared` (`local`) memory of size `(TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET) * (TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET)`.
   - Let the `work items` (threads) in a `work group` load their `input` elements into `shared` (`local`) memory.
     - For border elements, we load the value `0` into `shared` (`local`) memory (zero-padding).
   - Synchronize `work items` (threads) to ensure each `work item` (thread) in a `work group` has loaded its element into `shared` (`local`) memory.
   - Compute the convolution as in the CPU solution, but now using `shared` (`local`) memory (input) and `constant` memory (filter).
   - Store the result in the `ouput` matrix.
5. Create a function `main(void)`
   - Declare and allocate memory for matrices `input`, `ouput`, and `filter` on the host (CPU).
   - Initialize matrix `input` with `DATA_HEIGHT * DATA_WIDTH` random floats on the host (CPU).
   - Initialize matrix `filter` with `weights` on the host (CPU).
     - Each weight is `1.0 / (FILTER_HEIGHT * FILTER_WIDTH)` (averaging filter).
   - Declare and allocate memory for matrices `input`, `ouput` and `filter` on the device (GPU).
     - Copy `input` matrix in host (CPU) memory to device (GPU) global memory.
     - Copy `filter` matrix in host (CPU) memory to `constant` device (GPU) memory.
   
   - Set kernel function arguments:

      ```c
      // Set kernel arguments
      cl_int data_height = DATA_HEIGHT;
      cl_int data_width = DATA_WIDTH;
      cl_int filter_height_offset = FILTER_HEIGHT_OFFSET;
      cl_int filter_width_offset = FILTER_WIDTH_OFFSET;
      cl_int filter_width = FILTER_WIDTH;
      int workgroup_height = TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET;
      int workgroup_width = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET;
      int shared_size = workgroup_height * workgroup_width * sizeof(float);
      clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
      clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
      clSetKernelArg(kernel, 2, sizeof(cl_mem), &d_filter);
      clSetKernelArg(kernel, 3, shared_size, NULL);
      clSetKernelArg(kernel, 4, sizeof(int), &data_height);
      clSetKernelArg(kernel, 5, sizeof(int), &data_width);
      clSetKernelArg(kernel, 6, sizeof(int), &filter_height_offset);
      clSetKernelArg(kernel, 7, sizeof(int), &filter_width_offset);
      clSetKernelArg(kernel, 8, sizeof(int), &filter_width);
      ```
   - Set kernel launch configuration:
     - Notice the 2D configuration here by declaring `localSize[2]` and `gloablSize[2]` as 2D arrays.

      ```c
      // Kernel launch configuration
      size_t localSize[2] = { workgroup_width, workgroup_height };
      size_t globalSize[2] = {
          ((data_width + workgroup_width - 1) / workgroup_width) * workgroup_width,
          ((data_height + workgroup_height - 1) / workgroup_height) * workgroup_height
      };
      ```
   - Launch kernel with launch configuration `globalSize` and `localSize`.
   - Measure the execution time for the `kernel function`.
   - Copy `output` matrix in device (GPU) memory to host (CPU) memory.
   - Print execution time and sample elements in matrices `input` and `output`.
   - Free memory allocated for matrices `input`, `output`, and `filter` on the host (CPU).
   - Free memory allocated for matrices `input`, `output`, and `filter` on the device (GPU).

In [62]:
%%writefile src/kernel.cl
__kernel void mykernel(
    __global const float *input,
    __global float *output,
    __constant const float *filter,
    __local float *shared,
    const int data_height,
    const int data_width,
    const int filter_height_offset,
    const int filter_width_offset,
    const int filter_width)
{
    int s_row = get_local_id(1);
    int s_col = get_local_id(0);
    
    int d_row = get_global_id(1);
    int d_col = get_global_id(0);
    
    int i_row = d_row - filter_height_offset;
    int i_col = d_col - filter_width_offset;

    if (d_col >= data_width || d_row >= data_height) return;

    if (i_row >= 0 && i_row < data_height && i_col >= 0 && i_col < data_width)
    {
        shared[s_row * get_local_size(0) + s_col] = input[i_row * data_width + i_col];
    }
    else
    {
        shared[s_row * get_local_size(0) + s_col] = 0.0f; // zero-padding
    }

    barrier(CLK_LOCAL_MEM_FENCE);

    float sum = 0.0f;
    for (int offset_row = -filter_height_offset; offset_row <= filter_height_offset; offset_row++)
    {
        for (int offset_col = -filter_width_offset; offset_col <= filter_width_offset; offset_col++)
        {
            int f_row = filter_height_offset + offset_row;
            int f_col = filter_width_offset + offset_col;
            int i_row = s_row + f_row;
            int i_col = s_col + f_col;
            
            if(i_row >= 0 && i_row < get_local_size(1) && i_col >= 0 && i_col < get_local_size(0))
            {
                sum += shared[i_row * get_local_size(0) + i_col] * filter[f_row * filter_width + f_col];
            }
        }
    }
    output[d_row * data_width + d_col] = sum;
}

Overwriting src/kernel.cl


In [63]:
%%writefile src/main.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include "utils.h"

// Number of data elements in each dimension
#define DATA_WIDTH 32
#define DATA_HEIGHT 32

// Number of filter elements in each dimension
#define FILTER_WIDTH 3
#define FILTER_HEIGHT 3

// Number of elements on each side of a centered filter in each dimension
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH / 2)
#define FILTER_HEIGHT_OFFSET (FILTER_HEIGHT / 2)

// Number of elements in local (shared) memory in each dimension
#define TILE_WIDTH_BASE 16
#define TILE_HEIGHT_BASE 16

int main(void)
{
    // Setup OpenCL
    cl_int err; cl_context context; cl_command_queue queue; cl_program program; cl_kernel kernel;
    setupOpenCL(&context, &queue, &program, &kernel);

    srand(0);
    //srand(time(NULL));
    
    // Declare variables
    float *h_input, *h_output, *h_filter; // host copies of input, output, filter
    int data_size = DATA_WIDTH * DATA_HEIGHT * sizeof(float);
    int filter_size = FILTER_WIDTH * FILTER_HEIGHT * sizeof(float);

    // Allocate space for host (CPU) copies of input, output, filter
    h_input = (float *)malloc(data_size);
    h_output = (float *)malloc(data_size);
    h_filter = (float *)malloc(filter_size);

    // Setup input values
    for(int row = 0; row < DATA_HEIGHT; row++)
    {
        for(int col = 0; col < DATA_WIDTH; col++)
        {
            h_input[row * DATA_WIDTH + col] = (float)rand() / RAND_MAX;
        }
    }

    // Setup filter
    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            h_filter[row * FILTER_WIDTH + col] = 1.0f / (FILTER_WIDTH * FILTER_HEIGHT);
        }
    }

    // Device buffers
    cl_mem d_input, d_output, d_filter;
    d_input = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, data_size, h_input, &err);
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY, data_size, NULL, &err);    
    d_filter = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, filter_size, h_filter, &err);

    // Set kernel arguments
    cl_int data_height = DATA_HEIGHT;
    cl_int data_width = DATA_WIDTH;
    cl_int filter_height_offset = FILTER_HEIGHT_OFFSET;
    cl_int filter_width_offset = FILTER_WIDTH_OFFSET;
    cl_int filter_width = FILTER_WIDTH;
    int workgroup_height = TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET;
    int workgroup_width = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET;
    int shared_size = workgroup_height * workgroup_width * sizeof(float);
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &d_input);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &d_output);
    clSetKernelArg(kernel, 2, sizeof(cl_mem), &d_filter);
    clSetKernelArg(kernel, 3, shared_size, NULL);
    clSetKernelArg(kernel, 4, sizeof(int), &data_height);
    clSetKernelArg(kernel, 5, sizeof(int), &data_width);
    clSetKernelArg(kernel, 6, sizeof(int), &filter_height_offset);
    clSetKernelArg(kernel, 7, sizeof(int), &filter_width_offset);
    clSetKernelArg(kernel, 8, sizeof(int), &filter_width);

    // Kernel launch configuration
    size_t localSize[2] = { workgroup_width, workgroup_height };
    size_t globalSize[2] = {
        ((data_width + workgroup_width - 1) / workgroup_width) * workgroup_width,
        ((data_height + workgroup_height - 1) / workgroup_height) * workgroup_height
    };

    // Enqueue kernel with timing event
    cl_event kernel_event;
    clEnqueueNDRangeKernel(queue, kernel, 2, NULL, globalSize, localSize, 0, NULL, &kernel_event);

    // Wait for kernel to finish and compute execution time
    clWaitForEvents(1, &kernel_event);
    cl_ulong time_start, time_end;
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_START, sizeof(time_start), &time_start, NULL);
    clGetEventProfilingInfo(kernel_event, CL_PROFILING_COMMAND_END, sizeof(time_end), &time_end, NULL);
    double elapsed_ms = (time_end - time_start) * 1e-6;  // Convert nanoseconds to milliseconds

    // Copy result back to host
    clEnqueueReadBuffer(queue, d_output, CL_TRUE, 0, data_size, h_output, 0, NULL, NULL);

    // Print measured calculation execution time
    printf("Calculation (%d elements, %dx%d filter) took %.2f ms\n", DATA_HEIGHT * DATA_WIDTH, FILTER_HEIGHT, FILTER_WIDTH, elapsed_ms);
   
    // Print out the FILTER_WIDTH number of elements in the two arrays
    printf("\nMatrix samples:\n");
    printf("h_input %-12s h_output\n", "");
    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_input[row * DATA_WIDTH + col]);
        }
        printf("%-3s","");
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_output[row * DATA_WIDTH + col]);
        }
        printf("\n");
    }

    // Cleanup
    free(h_input);
    free(h_output);
    free(h_filter);
    clReleaseMemObject(d_input);
    clReleaseMemObject(d_output);
    clReleaseMemObject(d_filter);
    clReleaseEvent(kernel_event);
    
    // Teardown OpenCL
    teardownOpenCL(&context, &queue, &program, &kernel);

    return 0;
}

Overwriting src/main.c


In [64]:
!{build_multi_file_command}
!{execute_command}

Calculation (1024 elements, 3x3 filter) took 0.01 ms

Matrix samples:
h_input              h_output
0.840 0.394 0.783    0.238 0.396 0.382 
0.613 0.296 0.638    0.328 0.527 0.568 
0.267 0.540 0.375    0.325 0.514 0.616 


In the output we see:
- The results are the same for the GPU solution as for the CPU solution.
- The execution time for the GPU solution is significantly fast than the CPU solution.

---
# 4. Cleanup
---

- Let's remove all files that have been created by this notebook.

In [65]:
import os, shutil

dirs = ["src", "include", "bin", ".vscode"]
files = ["kernel.cl", "main.c", "main.exe"]

for d in dirs:
    if os.path.exists(d):
        shutil.rmtree(d)

for f in files:
    if os.path.exists(f):
        os.remove(f)